<a href="https://colab.research.google.com/github/luisbmonroyj/C4AG3E1_frontendVue/blob/main/NLPBasic_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 0: Acceso a los Datasets

Antes de comenzar con el preprocesamiento y modelado, necesitamos asegurarnos de tener acceso a los dos datasets principales que utilizaremos en este ejercicio:

1.  **Dataset de Reseñas de IMDb (`imdb_reviews`):**
    *   **Fuente:** Este dataset se carga directamente desde el catálogo de TensorFlow Datasets (`tfds`). TensorFlow se encarga de descargar y preparar los datos por nosotros la primera vez que se solicita.
    *   **Cómo lo accedemos:** Utilizamos la función `tfds.load('imdb_reviews', ...)`. Ya hemos ejecutado una celda (o la ejecutaremos si aún no lo has hecho) que descarga y carga este dataset en la variable `dataset`.
    *   **Contenido principal:** Texto de la reseña y una etiqueta de sentimiento binaria (0 o 1).

2.  **Dataset de Metadatos de Películas de IMDb (`movie_metadata.csv`):**
    *   **Fuente:** Este dataset proviene de Kaggle (`kevalm/movie-imbd-dataset`). Lo descargamos a nuestro entorno de Colab utilizando la API de Kaggle.
    *   **Cómo lo accedemos:** Primero, instalamos la librería `kaggle` y configuramos nuestras credenciales (idealmente guardadas en Google Drive). Luego, usamos el comando `!kaggle datasets download` para obtener el archivo zip, y finalmente lo descomprimimos. Ya hemos generado (o generaremos) las celdas para este proceso. Una vez descomprimido, el archivo `movie_metadata.csv` se carga en un DataFrame de pandas usando `pd.read_csv()`. Ya hemos ejecutado una celda que carga este archivo en la variable `df_movie_metadata`.
    *   **Contenido principal:** Metadatos sobre películas como título, director, géneros, presupuesto, puntuación de IMDb, número de votos, etc. (pero no el texto completo de las reseñas individuales).

Tener estos dos datasets disponibles en nuestras variables (`dataset` para las reseñas y `df_movie_metadata` para los metadatos) nos permite proceder con los siguientes pasos del plan.

### Cargar el dataset de reseñas de IMDb con TensorFlow Datasets (Paso 0)

In [ ]:
import tensorflow_datasets as tfds

# Cargar el dataset de reseñas de IMDb
# Esto descargará y preparará los datos si no se ha hecho antes.
# Usamos split='train' y as_supervised=True para obtener (texto, etiqueta).
try:
    dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
    print("Dataset 'imdb_reviews' cargado exitosamente en el Paso 0.")
    # Opcional: Mostrar información básica para confirmar
    # print(info)
    # print("\nPrimeros 5 ejemplos del dataset:")
    # for example, label in dataset.take(1):
    #     print(f"Texto: {example.numpy()}")
    #     print(f"Etiqueta: {label.numpy()}")
    #     print("-" * 20)
except Exception as e:
    print(f"Error al cargar el dataset 'imdb_reviews' en el Paso 0: {e}")
    print("Asegúrate de que tienes conexión a internet y suficiente espacio, y vuelve a intentar ejecutar esta celda.")

### Descargar dataset de Kaggle (Paso 0)

In [ ]:
# Instalar la biblioteca Kaggle (si no está instalada)
!pip install kaggle

### Montar Google Drive y configurar credenciales de Kaggle (Paso 0)

In [ ]:
from google.colab import drive
import os

# Montar Google Drive
drive_path = '/content/drive'
if not os.path.exists(drive_path):
    print("Montando Google Drive...")
    drive.mount(drive_path)
else:
    print("Google Drive ya está montado.")

# Define la ruta donde tienes o donde quieres guardar tu archivo kaggle.json en Google Drive
# Por ejemplo: '/content/drive/MyDrive/Kaggle_Credentials/kaggle.json'
# Asegúrate de que este archivo existe en esta ubicación en tu Drive.
kaggle_json_path_in_drive = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1/kaggle.json'

# Verifica si el archivo kaggle.json existe en la ruta especificada en Drive
if not os.path.exists(kaggle_json_path_in_drive):
    print(f"Error: No se encontró el archivo kaggle.json en la ruta especificada: {kaggle_json_path_in_drive}")
    print("Por favor, asegúrate de que tu archivo kaggle.json está en esa ubicación en tu Google Drive.")
    # Puedes subirlo manualmente a esa carpeta en tu Drive o usar el siguiente código para subirlo AHORA a Colab y luego copiarlo a Drive
    # from google.colab import files
    # uploaded = files.upload()
    # # Asumiendo que subiste 'kaggle.json'
    # !mkdir -p $(dirname "{kaggle_json_path_in_drive}")
    # !cp kaggle.json "{kaggle_json_path_in_drive}"
else:
    print(f"Archivo kaggle.json encontrado en: {kaggle_json_path_in_drive}")
    # Configurar la variable de entorno KAGGLE_CONFIG_DIR
    # Apuntamos KAGGLE_CONFIG_DIR al directorio que contiene kaggle.json
    kaggle_config_dir = os.path.dirname(kaggle_json_path_in_drive)
    os.environ['KAGGLE_CONFIG_DIR'] = kaggle_config_dir
    print(f"Variable de entorno KAGGLE_CONFIG_DIR configurada a: {os.environ.get('KAGGLE_CONFIG_DIR')}")
    print("Ahora Kaggle buscará las credenciales en tu Google Drive.")

### Descargar el dataset de Kaggle (Paso 0)

In [ ]:
# Descargar el dataset usando el identificador
# El comando buscará kaggle.json en el directorio especificado por KAGGLE_CONFIG_DIR
!kaggle datasets download -d kevalm/movie-imbd-dataset

# Listar los archivos descargados (debería ser un archivo zip en el directorio actual)
!ls

### Descomprimir el archivo del dataset (Paso 0)

In [ ]:
import zipfile
import os

# Definir el nombre del archivo zip (ajustar si es diferente)
# Esto asumirá que el archivo se descargó en el directorio actual
zip_file_name = 'movie-imbd-dataset.zip'

# Definir el directorio de destino para la descompresión
destination_dir = 'imdb_dataset_from_kaggle'
os.makedirs(destination_dir, exist_ok=True)

# Verificar si el archivo zip existe antes de intentar descomprimir
if os.path.exists(zip_file_name):
    # Descomprimir el archivo zip
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall(destination_dir)

    print(f"Dataset descomprimido en la carpeta: {destination_dir}")

    # Listar los archivos descomprimidos
    print(f"Contenido de la carpeta '{destination_dir}':")
    !ls {destination_dir}
else:
    print(f"Error: El archivo zip '{zip_file_name}' no fue encontrado en el directorio actual.")
    print("Por favor, verifica si la descarga de Kaggle fue exitosa.")

## Paso 1: Preparar el dataset `imdb_reviews` para entrenamiento

El objetivo de este paso es convertir las reseñas de texto en un formato numérico que pueda ser utilizado por un modelo de Machine Learning. Esto implica varias sub-etapas de preprocesamiento.

### Sub-etapa 1.1: Limpieza básica del texto

Las reseñas de texto a menudo contienen ruido como etiquetas HTML (`<br />`), caracteres especiales, puntuación, etc., que no aportan valor para la clasificación de sentimiento e incluso pueden confundir al modelo. Vamos a realizar una limpieza básica.

In [ ]:
import re
import string
import tensorflow as tf # Importamos TensorFlow para trabajar con el dataset

# Asegurarnos de que el dataset esté cargado (si no lo estaba ya)
# Si la celda anterior de carga falló o se interrumpió, ejecutar esta parte es crucial
try:
    dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
    print("Dataset 'imdb_reviews' cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
    print("Por favor, ejecuta la celda de carga del dataset 'imdb_reviews' si aún no lo has hecho o si falló.")
    # Si el dataset no se carga, las siguientes celdas darán error.
    # En un escenario real, aquí podrías detener la ejecución o manejar el error de otra forma.


# Función de limpieza de texto
def clean_text(text):
    # Convertir a minúsculas
    text = tf.strings.lower(text)
    # Eliminar etiquetas HTML (como <br />)
    text = tf.strings.regex_replace(text, '<br />', ' ')
    # Eliminar puntuación y mantener solo letras, números y espacios
    text = tf.strings.regex_replace(text, '[%s]' % re.escape(string.punctuation), '')
    # Eliminar números (opcional, a veces los números pueden ser relevantes)
    # text = tf.strings.regex_replace(text, '[0-9]+', '')
    # Eliminar espacios extra
    text = tf.strings.regex_replace(text, '\\s+', ' ')
    # Eliminar espacios al inicio y final
    text = tf.strings.strip(text)
    return text

# Aplicar la función de limpieza a los ejemplos del dataset
# Usaremos map para aplicar la función a cada elemento del dataset de TensorFlow
dataset_cleaned = dataset.map(lambda text, label: (clean_text(text), label))

# Mostrar algunos ejemplos limpios para verificar
print("\nPrimeros 5 ejemplos del dataset después de la limpieza:")
for example, label in dataset_cleaned.take(5):
    print(f"Texto limpio: {example.numpy()}")
    print(f"Etiqueta: {label.numpy()}")
    print("-" * 20)

### Sub-etapa 1.1: Limpieza básica del texto, Stemming y Lematización

Las reseñas de texto a menudo contienen ruido como etiquetas HTML (`<br />`), caracteres especiales, puntuación, etc., que no aportan valor para la clasificación de sentimiento e incluso pueden confundir al modelo. Además, palabras con la misma raíz pero diferentes terminaciones (como "corriendo", "corre", "corrió") pueden tratarse como la misma palabra base para reducir la dimensionalidad y capturar mejor el significado. Para esto, usamos **Stemming** y **Lematización**.

*   **Limpieza básica:** Eliminará ruido irrelevante.
*   **Stemming:** Reduce las palabras a su raíz o "stem" (ej: "running" -> "run"). Es un proceso más rudimentar que corta sufijos.
*   **Lematización:** Reduce las palabras a su forma base o "lema" (ej: "running" -> "run", "better" -> "good"). Es un proceso más sofisticado que considera el vocabulario y el análisis morfológico para llegar a la forma base correcta de una palabra.

### Sub-etapa 1.1.1: Instalar NLTK y descargar recursos

In [ ]:
# Instalar NLTK
!pip install nltk

In [ ]:
import nltk

# Descargar recursos necesarios para Stemming y Lematización
nltk.download('punkt') # Para tokenización
nltk.download('wordnet') # Para lematización
nltk.download('omw-1.4') # Open Multilingual Wordnet (complemento para wordnet)
nltk.download('averaged_perceptron_tagger') # Para identificar la parte del habla (útil para lematización)
nltk.download('punkt_tab') # Recurso adicional para tokenización, necesario según el error
nltk.download('averaged_perceptron_tagger_eng') # Recurso específico para POS tagging en inglés, necesario según el último error

# NOTA: Ejecuta las líneas de nltk.download() una vez si no tienes los recursos.
# Las dejo comentadas para evitar descargas repetidas si ya las hiciste.

### Sub-etapa 1.1.2: Aplicar Stemming y Lematización (en una muestra)

Vamos a demostrar cómo aplicar Stemming y Lematización. Ten en cuenta que aplicar esto directamente a todo el dataset de TensorFlow de manera eficiente requiere envolver las funciones de NLTK para que funcionen con `tf.py_function` o procesar el dataset de otra forma (por lotes convertidos a NumPy/listas). Para este ejemplo, lo haremos en una pequeña muestra para entender el concepto.

In [ ]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

# Inicializar Stemmer y Lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Helper function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNetLemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # Default to Noun if tag not found


# Ejemplo de texto limpio (tomado de la salida anterior)
sample_text_bytes = b'this was an absolutely terrible movie dont be lured in by christopher walken or michael ironside both are great actors but this must simply be their worst role in history even their great acting could not redeem this movies ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the columbian rebels were making their cases for revolutions maria conchita alonso appeared phony and her pseudolove affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actors like christopher walkens good name i could barely sit through it'

# Decodificar el texto de bytes a string
sample_text = sample_text_bytes.decode('utf-8')

print(f"Texto original (limpio): {sample_text}")

# Tokenizar el texto
tokens = word_tokenize(sample_text)
print(f"\nTokens: {tokens[:20]}...") # Mostrar los primeros tokens

# Aplicar Stemming
stemmed_tokens = [stemmer.stem(word) for word in tokens]
print(f"\nTokens después de Stemming: {stemmed_tokens[:20]}...") # Mostrar los primeros tokens

# Aplicar Lematización
lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]
print(f"\nTokens después de Lematización: {lemmatized_tokens[:20]}...") # Mostrar los primeros tokens

# Nota técnica:
# - Stemming es más rápido pero menos preciso (solo corta).
# - Lematización es más lento pero más preciso (usa vocabulario y POS).
# - Para grandes datasets en TF, considera usar tf.py_function con NLTK o bibliotecas nativas de TF si están disponibles para estas tareas.
# - Puedes combinar la limpieza básica con Stemming/Lematización en una sola función de procesamiento.

### Sub-etapa 1.1.3: Visualizar diferencias con Word Clouds

In [ ]:
# Instalar la librería wordcloud
!pip install wordcloud matplotlib

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np # Necesario para manejar el texto en bytes de tf.data

# Usaremos el mismo sample_text_bytes de la celda anterior para demostrar
# sample_text_bytes ya está definido en la celda anterior (78276323)
# Si ejecutas esta celda por separado, asegúrate de que sample_text_bytes esté definido.

# Decodificar el texto de bytes a string (limpio, sin stemming/lemmatization aún)
sample_text_cleaned = sample_text_bytes.decode('utf-8')

# --- Aplicar Stemming y preparar texto para Word Cloud ---
# Asegurarnos de que stemmer y word_tokenize estén inicializados (si no se han ejecutado antes)
try:
    stemmer = PorterStemmer()
    tokens = word_tokenize(sample_text_cleaned)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    text_stemmed = " ".join(stemmed_tokens)
except NameError:
     print("Stemmer o tokenizer no inicializados. Ejecuta la celda 1.1.2 primero.")
     text_stemmed = "" # Evitar error si no se puede generar


# --- Aplicar Lematización y preparar texto para Word Cloud ---
# Asegurarnos de que lemmatizer, word_tokenize, wordnet y nltk.pos_tag estén disponibles
try:
    lemmatizer = WordNetLemmatizer()
    # get_wordnet_pos function is defined in cell 78276323
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]
    text_lemmatized = " ".join(lemmatized_tokens)
except NameError:
    print("Lemmatizer o recursos de NLTK no inicializados. Ejecuta la celda 1.1.2 primero.")
    text_lemmatized = "" # Evitar error si no se puede generar
except LookupError:
    print("Recursos de NLTK para lematización no descargados. Ejecuta la celda 1.1.1 primero.")
    text_lemmatized = ""


# --- Generar Word Clouds ---

# Configuración básica para las word clouds
wordcloud_config = {
    'width': 800,
    'height': 400,
    'background_color': 'black', # Cambiado a oscuro
    'max_words': 100,
    'contour_color': 'steelblue'
}

plt.figure(figsize=(15, 5))

# Word Cloud para Texto Limpio
if sample_text_cleaned:
    wordcloud_cleaned = WordCloud(**wordcloud_config).generate(sample_text_cleaned)
    plt.subplot(1, 3, 1)
    plt.imshow(wordcloud_cleaned, interpolation='bilinear')
    plt.axis('off')
    plt.title('Texto Limpio')
else:
     plt.subplot(1, 3, 1)
     plt.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center')
     plt.axis('off')
     plt.title('Texto Limpio (Error)')


# Word Cloud para Stemming
if text_stemmed:
    wordcloud_stemmed = WordCloud(**wordcloud_config).generate(text_stemmed)
    plt.subplot(1, 3, 2)
    plt.imshow(wordcloud_stemmed, interpolation='bilinear')
    plt.axis('off')
    plt.title('Texto con Stemming')
else:
     plt.subplot(1, 3, 2)
     plt.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center')
     plt.axis('off')
     plt.title('Texto con Stemming (Error)')


# Word Cloud para Lematización
if text_lemmatized:
    wordcloud_lemmatized = WordCloud(**wordcloud_config).generate(text_lemmatized)
    plt.subplot(1, 3, 3)
    plt.imshow(wordcloud_lemmatized, interpolation='bilinear')
    plt.axis('off')
    plt.title('Texto con Lematización')
else:
     plt.subplot(1, 3, 3)
     plt.text(0.5, 0.5, 'No data', horizontalalignment='center', verticalalignment='center')
     plt.axis('off')
     plt.title('Texto con Lematización (Error)')


plt.tight_layout()
plt.show()

# Recomendación/Hack:
# Las Word Clouds son excelentes para visualizaciones rápidas de la frecuencia de palabras.
# Notarás que el stemming puede crear "palabras" que no son reales (ej. 'movi' en lugar de 'movie'),
# mientras que la lematización intenta mantener la forma base real ('movie').
# Esto ilustra la diferencia de precisión entre las dos técnicas.

### Sub-etapa 1.2: Tokenización y Vectorización (Usando TF-IDF)

Después de limpiar y normalizar el texto (aplicando Lematización), el siguiente paso es dividir las reseñas en palabras individuales (Tokenización) y luego convertir estas palabras en vectores numéricos que puedan ser entendidos por un modelo. Una técnica común es TF-IDF (Frecuencia de Término - Frecuencia Inversa de Documento).

**Tokenización:** El proceso de dividir una secuencia de texto en piezas más pequeñas llamadas tokens (normalmente palabras).
**Vectorización (TF-IDF):** Asigna un peso a cada token basado en qué tan frecuentemente aparece en una reseña (Frecuencia de Término) y qué tan raro es el término en todo el conjunto de reseñas (Frecuencia Inversa de Documento). Esto ayuda a resaltar las palabras que son más importantes para una reseña específica en comparación con el corpus general.

In [ ]:
# Nota: Para aplicar la limpieza y lematización a todo el dataset de TF de manera eficiente,
# necesitaríamos integrar las funciones de NLTK con tf.py_function o usar alternativas nativas de TF/Keras.
# Para simplificar este ejemplo y continuar con la vectorización, vamos a simular
# que tenemos un conjunto de textos limpios y lematizados listos para vectorizar.

# En un flujo de trabajo completo con TF.data, aplicarías las funciones de limpieza/lematización
# usando dataset.map() con tf.py_function o Keras Preprocessing layers (si usas una capa de lematización
# personalizada o si usas Keras's built-in text processing which might not have native lemmatization).

# --- Simulación de textos limpios y lematizados ---
# Tomamos una muestra pequeña del dataset limpio/lematizado si ya ejecutaste esa parte,
# o usamos los textos de ejemplo de la Sub-etapa 1.1.2 para demostrar.

# Si ya tienes dataset_cleaned de la celda b448b791, puedes usarlo:
# sample_texts = [example.numpy().decode('utf-8') for example, label in dataset_cleaned.take(10)]

# Si no, usamos los textos de ejemplo de la sub-etapa 1.1.2 (celda 78276323):
sample_text_cleaned_demo = b'this was an absolutely terrible movie dont be lured in by christopher walken or michael ironside both are great actors but this must simply be their worst role in history even their great acting could not redeem this movies ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the columbian rebels were making their cases for revolutions maria conchita alonso appeared phony and her pseudolove affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actors like christopher walkens good name i could barely sit through it'.decode('utf-8')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk
import re
import string
import numpy as np # Import numpy for array handling
import pandas as pd # ¡Importar pandas!

# Asegurarse de que los recursos de NLTK estén descargados si no lo están
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
    nltk.data.find('taggers/averaged_perceptron_tagger')
    # nltk.data.find('tokenizers/punkt_tab') # Optional based on previous errors
    # nltk.data.find('taggers/averaged_perceptron_tagger_eng') # Optional based on previous errors
except LookupError:
    print("Recursos de NLTK no encontrados. Por favor, ejecuta la celda de descarga de recursos de NLTK (Sub-etapa 1.1.1) primero.")


# Helper function to get the part of speech tag for lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNetLemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # Default to Noun if tag not found

# Combinar limpieza y lematización en una función (aplicada a un string)
def clean_and_lemmatize(text):
    # Limpieza básica (similar a la anterior pero para string de Python)
    text = text.lower()
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenización
    tokens = word_tokenize(text)

    # Lematización
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens]

    return " ".join(lemmatized_tokens)


# Aplicar la limpieza y lematización a una muestra de textos (convertidos a lista de strings)
# Para este ejemplo, tomamos 10 textos del dataset original, los convertimos a string, limpiamos y lematizamos.
# NOTA: Aplicar NLTK a todo el dataset de TF de esta forma NO es eficiente. Es solo para demostración.
sample_texts_raw = [example.numpy().decode('utf-8') for example, label in dataset.take(10)] # Usamos el dataset original
sample_texts_processed = [clean_and_lemmatize(text) for text in sample_texts_raw]


# --- Vectorización usando TF-IDF ---
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar el vectorizador TF-IDF
# max_features limita el vocabulario para manejar el tamaño
tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Considerar las 5000 palabras más frecuentes/importantes

# Ajustar el vectorizador a los textos procesados y transformar los textos en una matriz TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_texts_processed)

# Obtener los nombres de las características (palabras)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Mostrar la matriz TF-IDF (para una muestra pequeña)
print("\nMatriz TF-IDF (primeros 10 textos, primeras 20 características):")
# Convertir la matriz dispersa a densa para mostrar (SOLO PARA MUESTRAS PEQUEÑAS)
tfidf_matrix_dense = tfidf_matrix.toarray()
display(pd.DataFrame(tfidf_matrix_dense[:10, :20], columns=feature_names[:20]))

# En un escenario real, trabajarías con la matriz dispersa para ahorrar memoria.

print(f"\nForma de la matriz TF-IDF (Número de textos, Número de características/palabras): {tfidf_matrix.shape}")
print(f"Número de características (palabras únicas en el vocabulario limitado): {len(feature_names)}")

# Recomendación:
# Para el dataset completo de IMDb (~25000 reseñas), TF-IDF con un vocabulario grande
# puede generar una matriz muy grande. Considera usar un max_features razonable.
# Alternativas como CountVectorizer también son posibles, pero TF-IDF suele dar mejores resultados
# porque pondera la importancia de las palabras.
# Embeddings de palabras (Word2Vec, GloVe, o de modelos como BERT) son una alternativa
# más moderna y potente a TF-IDF para capturar significado semántico. Las exploraremos después.

### Sub-etapa 1.1.4: Eliminar Stop Words

In [ ]:
import nltk

# Descargar la lista de stop words
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("Lista de Stop words de NLTK descargada.")

### Sub-etapa 1.1.5: Integrar Limpieza, Lematización y Eliminación de Stop Words

Ahora vamos a modificar nuestra función de limpieza para incluir la eliminación de stop words y asegurarnos de que la lematización se aplique sobre los tokens que no son stop words.

In [ ]:
import re
import string
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
import nltk # Asegurarse de que nltk esté importado para get_wordnet_pos y descargas

# Asegurarse de que el dataset esté cargado
try:
    dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
    print("Dataset 'imdb_reviews' cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
    print("Por favor, ejecuta la celda de carga del dataset 'imdb_reviews' si aún no lo has hecho o si falló.")

# Asegurarse de que los recursos de NLTK y stop words estén descargados si no lo están
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
    nltk.data.find('taggers/averaged_perceptron_tagger')
    nltk.data.find('corpora/stopwords')
    # Opcionales según errores anteriores:
    # nltk.data.find('tokenizers/punkt_tab')
    # nltk.data.find('taggers/averaged_perceptron_tagger_eng')
except LookupError:
    print("Recursos de NLTK o stop words no encontrados. Por favor, ejecuta las celdas de descarga.")


# Obtener la lista de stop words en inglés
stop_words = set(stopwords.words('english'))

# Inicializar Lemmatizer y POS tagger (recursos ya descargados)
lemmatizer = WordNetLemmatizer()
# get_wordnet_pos function is defined in a previous cell (78276323),
# assuming it's available in the global scope or define it here again for clarity
def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNetLemmatizer"""
    # Requires 'averaged_perceptron_tagger' and 'punkt' resources
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN) # Default to Noun if tag not found


# Función de limpieza, tokenización, eliminación de stop words y lematización
# Esta función trabajará en un string de Python, no directamente en un tf.Tensor
# Para aplicarla a tf.data.Dataset, necesitaríamos usar tf.py_function
def clean_tokenize_lemmatize_stopwords(text):
    # Decodificar si es necesario (si el input es bytes)
    if isinstance(text, tf.Tensor):
        text = text.numpy().decode('utf-8')
    elif isinstance(text, bytes):
         text = text.decode('utf-8')

    # Limpieza básica
    text = text.lower()
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenización
    tokens = word_tokenize(text)

    # Eliminar Stop Words y Lematizar
    processed_tokens = []
    for word in tokens:
        if word not in stop_words: # Eliminar stop words
            # Aplicar lematización solo a palabras no stop words
            lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
            processed_tokens.append(lemma)

    return " ".join(processed_tokens)

# --- Aplicar esta función a una muestra del dataset para demostración ---
# NOTA: Aplicar NLTK/Python functions a TF.data.Dataset es más eficiente usando tf.py_function
# o procesando por lotes. Esto es solo para demostrar la función.
sample_texts_raw_demo = [example.numpy() for example, label in dataset.take(5)] # Tomar 5 ejemplos en bytes
sample_texts_processed_demo = [clean_tokenize_lemmatize_stopwords(text) for text in sample_texts_raw_demo]

print("Primeros 5 ejemplos después de limpieza, eliminación de stop words y lematización:")
for i, text in enumerate(sample_texts_processed_demo):
    print(f"Ejemplo {i+1}: {text}")
    print("-" * 20)

# Recomendación para usar con tf.data.Dataset:
# Envuelve la función clean_tokenize_lemmatize_stopwords en tf.py_function
# para aplicarla eficientemente al dataset completo:
# dataset_processed = dataset.map(lambda text, label: (tf.py_function(func=clean_tokenize_lemmatize_stopwords, inp=[text], Tout=tf.string), label))
# Sin embargo, tf.py_function puede ser más lento que las operaciones nativas de TF/Keras.
# Para pipelines eficientes en TF, considera usar Keras Preprocessing layers si es posible,
# aunque la lematización avanzada con POS tagging como la de NLTK no está nativamente en Keras.

### Sub-etapa 1.3: Manejo de Datos Faltantes en `df_movie_metadata`

El dataset de metadatos de Kaggle (`df_movie_metadata`) contiene valores faltantes (`NaN`) que necesitamos abordar antes de poder utilizarlo plenamente en los pasos posteriores.

### Identificar valores faltantes

In [ ]:
import pandas as pd # Asegurarse de que pandas esté importado

# Inicializar df_movie_metadata a None para evitar NameError si no se carga previamente
df_movie_metadata = None

# Asegurarnos de que df_movie_metadata esté cargado
try:
    # Intentar usar la variable existente si ya fue cargada en una celda anterior
    # Verificamos si la variable existe en el ámbito global y si es un DataFrame
    if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame):
         print("Utilizando el DataFrame df_movie_metadata cargado previamente.")
    else:
        # Si la variable no existe o no es un DataFrame, intentar cargar el archivo CSV
        print("La variable df_movie_metadata no fue encontrada o no es un DataFrame. Intentando cargar el archivo CSV.")
        csv_file_path_metadata = '/content/imdb_dataset_from_kaggle/movie_metadata.csv'
        df_movie_metadata = pd.read_csv(csv_file_path_metadata)
        print(f"Dataset '{csv_file_path_metadata}' cargado exitosamente.")

except FileNotFoundError:
    print(f"Error: El archivo CSV '{csv_file_path_metadata}' no fue encontrado.")
    print("Por favor, verifica la ruta y asegúrate de haber descomprimido el dataset de Kaggle.")
    df_movie_metadata = None # Asegurarse de que es None si la carga falla
except Exception as e:
    print(f"Error inesperado al cargar df_movie_metadata: {e}")
    df_movie_metadata = None


if df_movie_metadata is not None:
    # Contar valores faltantes por columna
    missing_values_count = df_movie_metadata.isnull().sum()

    # Mostrar las columnas con valores faltantes (filtrando las que tienen 0)
    missing_values_count = missing_values_count[missing_values_count > 0]

    print("\nNúmero de valores faltantes por columna en df_movie_metadata:")
    display(missing_values_count.sort_values(ascending=False))

    # Opcional: Mostrar el porcentaje de valores faltantes
    print("\nPorcentaje de valores faltantes por columna:")
    missing_values_percent = (df_movie_metadata.isnull().sum() / len(df_movie_metadata)) * 100
    missing_values_percent = missing_values_percent[missing_values_percent > 0]
    display(missing_values_percent.sort_values(ascending=False))
else:
    print("\nNo se pudo cargar el DataFrame df_movie_metadata. No se puede proceder con el análisis de valores faltantes.")

### Manejar valores faltantes: Eliminar filas con datos nulos

In [ ]:
# Asegurarnos de que df_movie_metadata esté cargado
try:
    # Intentar usar la variable existente si ya fue cargada
    print("Utilizando el DataFrame df_movie_metadata cargado previamente.")
except NameError:
    # Si la variable no existe, intentar cargar el archivo CSV nuevamente
    print("La variable df_movie_metadata no fue encontrada. Intentando cargar el archivo CSV.")
    try:
        csv_file_path_metadata = '/content/imdb_dataset_from_kaggle/movie_metadata.csv'
        df_movie_metadata = pd.read_csv(csv_file_path_metadata)
        print(f"Dataset '{csv_file_path_metadata}' cargado exitosamente.")
    except FileNotFoundError:
        print(f"Error: El archivo CSV '{csv_file_path_metadata}' no fue encontrado.")
        print("Por favor, verifica la ruta y asegúrate de haber descomprimido el dataset de Kaggle.")
        df_movie_metadata = None # Establecer como None para evitar errores posteriores

if df_movie_metadata is not None:
    print(f"Tamaño original del DataFrame: {df_movie_metadata.shape}")

    # Eliminar filas que contengan *al menos un* valor faltante
    df_movie_metadata_cleaned = df_movie_metadata.dropna()

    print(f"Tamaño del DataFrame después de eliminar filas con valores faltantes: {df_movie_metadata_cleaned.shape}")

    # Mostrar cuántas filas fueron eliminadas
    rows_dropped = df_movie_metadata.shape[0] - df_movie_metadata_cleaned.shape[0]
    print(f"Número de filas eliminadas debido a valores faltantes: {rows_dropped}")

    # Mostrar si aún quedan valores faltantes (debería ser 0)
    print("\nVerificación de valores faltantes después de la eliminación:")
    display(df_movie_metadata_cleaned.isnull().sum().sum()) # Suma total de NaN en todo el DataFrame

    # --- Alternativa (comentada): Imputación ---
    # Si en lugar de eliminar, quisieras imputar, podrías hacer algo como esto:
    # df_movie_metadata_imputed = df_movie_metadata.copy() # Trabajar en una copia
    # # Imputar columnas numéricas con la mediana
    # for col in df_movie_metadata_imputed.select_dtypes(include=np.number).columns:
    #     if df_movie_metadata_imputed[col].isnull().any():
    #         median_val = df_movie_metadata_imputed[col].median()
    #         df_movie_metadata_imputed[col].fillna(median_val, inplace=True)
    # # Imputar columnas categóricas con la moda (ejemplo básico, puede requerir más lógica)
    # for col in df_movie_metadata_imputed.select_dtypes(include='object').columns:
    #      if df_movie_metadata_imputed[col].isnull().any():
    #          mode_val = df_movie_metadata_imputed[col].mode()[0] # mode() puede devolver múltiples valores
    #          df_movie_metadata_imputed[col].fillna(mode_val, inplace=True)
    # print("\n(Alternativa) Verificación de valores faltantes después de imputación (en df_movie_metadata_imputed):")
    # display(df_movie_metadata_imputed.isnull().sum().sum())


    # Ahora, continuaremos trabajando con df_movie_metadata_cleaned para los siguientes pasos
    df_movie_metadata = df_movie_metadata_cleaned # Reemplazamos el DataFrame original con la versión limpia
    print("\nEl DataFrame 'df_movie_metadata' ahora contiene solo filas sin valores faltantes.")

### Sub-etapa 1.2.1: Tokenización y Estadísticas

Después de la limpieza básica y lematización, el siguiente paso es tokenizar el texto y analizar algunas estadísticas.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import nltk
import re
import string
import collections # Para contar la frecuencia de los tokens
import numpy as np # Importar numpy
import pandas as pd # Importar pandas para mostrar resultados en tabla

# Asegurar que los recursos de NLTK y stop words estén descargados
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
    nltk.data.find('taggers/averaged_perceptron_tagger')
    nltk.data.find('corpora/stopwords')
    # Optional based on previous errors:
    # nltk.data.find('tokenizers/punkt_tab')
    # nltk.data.find('taggers/averaged_perceptron_tagger_eng')
except LookupError:
    print("Recursos de NLTK o stop words no encontrados. Ejecuta las celdas de descarga de la Sub-etapa 1.1.1 y 1.1.4.")
    # Esto no detiene la ejecución, pero la función clean_tokenize_lemmatize_stopwords fallará si no se descargan.


# Asegurar que el dataset esté cargado
try:
    # Si la variable 'dataset' ya existe, la usamos.
    print("Utilizando el dataset 'imdb_reviews' cargado previamente.")
except NameError:
    # Si no, intentamos cargarlo.
    print("Dataset 'imdb_reviews' no encontrado. Intentando cargarlo.")
    try:
        dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
        print("Dataset 'imdb_reviews' cargado exitosamente.")
    except Exception as e:
        print(f"Error al cargar el dataset: {e}")
        print("Por favor, ejecuta la celda de carga del dataset 'imdb_reviews' (Paso 0).")
        dataset = None # Asegurar que dataset es None si la carga falla

if dataset is not None:
    # Obtener la lista de stop words en inglés
    try:
      stop_words = set(stopwords.words('english'))
    except LookupError:
      print("Recurso 'stopwords' de NLTK no encontrado. Ejecuta la celda de descarga de stop words (Sub-etapa 1.1.4).")
      stop_words = set() # Usar un set vacío para evitar errores si no se descargan


    # Inicializar Lemmatizer y POS tagger
    lemmatizer = WordNetLemmatizer()
    # Redefinir la función get_wordnet_pos para asegurar que esté disponible y maneje errores de recursos
    def get_wordnet_pos(word):
        """Map POS tag to first character used by WordNetLemmatizer, handles LookupError"""
        try:
            tag = nltk.pos_tag([word])[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
            return tag_dict.get(tag, wordnet.NOUN) # Default to Noun
        except LookupError:
            # Si falta el recurso 'averaged_perceptron_tagger', retornar Noun por defecto
            print("Recurso 'averaged_perceptron_tagger' de NLTK no encontrado. La lematización será menos precisa.")
            return wordnet.NOUN
        except IndexError:
            # Manejar casos donde pos_tag no devuelve la estructura esperada (ej. palabra vacía)
            return wordnet.NOUN


    # Función de limpieza, tokenización, eliminación de stop words y lematización (para un string)
    def clean_tokenize_lemmatize_stopwords(text_input):
        # Si el input es un tensor de TF (como cuando se itera con .batch(1)),
        # extraer el valor numpy y decodificar el primer (y único) elemento.
        if isinstance(text_input, tf.Tensor):
             # Asegurar que el tensor no esté vacío y contenga bytes
             if tf.size(text_input) > 0 and text_input.dtype == tf.string:
                 # Acceder al elemento (batch de tamaño 1) y decodificar
                 text = text_input.numpy()[0].decode('utf-8', errors='ignore') # Usar errors='ignore' para evitar problemas con caracteres
             else:
                 return [] # Retornar lista vacía si el tensor está vacío o no es string
        elif isinstance(text_input, bytes):
             text = text_input.decode('utf-8', errors='ignore')
        else: # Asumir que ya es un string
             text = str(text_input) # Convertir a string explícitamente

        # Limpieza básica
        text = text.lower()
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        # Tokenización
        try:
            tokens = word_tokenize(text)
        except LookupError:
            print("Recurso 'punkt' de NLTK no encontrado. No se realizará tokenización.")
            return [] # Retornar lista vacía si falta el recurso de tokenización

        # Eliminar Stop Words y Lematizar
        processed_tokens = []
        for word in tokens:
            # Verificar si word es una cadena no vacía antes de procesar
            if word and word not in stop_words: # Eliminar stop words
                # Aplicar lematización solo a palabras no stop words
                # get_wordnet_pos maneja su propio LookupError ahora
                lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
                processed_tokens.append(lemma)

        return processed_tokens # Retornamos una lista de tokens

    # --- Procesar el dataset completo (¡puede tardar!) ---
    # NOTA: Aplicar NLTK/Python functions a un TF.data.Dataset completo de esta forma (iterando)
    # NO es la forma más eficiente para pipelines de entrenamiento en TF.
    # Para entrenamiento, idealmente usarías tf.py_function o Keras Preprocessing layers.
    # Hacemos esto aquí solo para obtener las estadísticas globales.

    all_tokens = []
    total_reviews = 0

    print("\nProcesando dataset y recolectando tokens (esto puede tardar varios minutos)...")

    # Iterar sobre el dataset. Usamos .batch(1) para obtener tensores de tamaño 1.
    # Convertir a numpy() para usar funciones Python/NLTK
    # Aumentar el tamaño del lote para una ligera mejora de eficiencia al iterar
    # Aunque para tf.py_function o Keras layers se usarían lotes más grandes.
    batch_size_for_stats = 32 # Procesar 32 reseñas a la vez para las estadísticas

    # Usar as_numpy_iterator() para iterar más eficientemente sobre lotes convertidos a numpy
    for text_batch, label_batch in dataset.batch(batch_size_for_stats).as_numpy_iterator():
        # text_batch es un array numpy de bytes strings
        for text_bytes in text_batch:
            tokens = clean_tokenize_lemmatize_stopwords(text_bytes) # Pasar los bytes directamente
            all_tokens.extend(tokens)

        total_reviews += len(text_batch) # Sumar el tamaño del lote al total
        if total_reviews % 1000 == 0:
            print(f"Procesadas {total_reviews} reseñas...")

    print(f"\nProcesamiento completo. Total de reseñas procesadas: {total_reviews}")

    # --- Calcular Estadísticas ---
    total_tokens = len(all_tokens)
    unique_tokens = len(set(all_tokens))
    token_counts = collections.Counter(all_tokens) # Frecuencia de cada token

    print(f"\nEstadísticas del Dataset después de Limpieza, Lematización y Eliminación de Stop Words:")
    print(f"Número total de tokens: {total_tokens}")
    print(f"Número de tokens únicos (tamaño del vocabulario): {unique_tokens}")

    print("\nTop 20 tokens más comunes:")
    display(pd.DataFrame(token_counts.most_common(20), columns=['Token', 'Frecuencia']))

    # Recomendación: El tamaño del vocabulario es un factor clave para la vectorización.
    # Un vocabulario muy grande puede hacer que las matrices TF-IDF sean enormes.
    # Podemos limitar el vocabulario a las N palabras más frecuentes en el siguiente paso.

else:
    print("\nNo se pudo procesar el dataset porque no fue cargado exitosamente.")

### Sub-etapa 1.2.2: Visualizar términos con alto TF-IDF en reseñas de muestra

In [ ]:
# Usaremos el tfidf_vectorizer que ajustamos previamente (en la celda 4704de53)
# sobre una muestra de textos procesados.

# Si la variable tfidf_vectorizer no está definida (por ejemplo, si reiniciaste
# el entorno sin ejecutar la celda 4704de53), necesitarás ejecutarla primero.
# También necesitamos la función clean_and_lemmatize de la celda 4704de53
# y el dataset 'dataset' de la celda 4469e4b8.

# Asegurarnos de que las variables necesarias estén disponibles
try:
    tfidf_vectorizer
    clean_and_lemmatize
    dataset
except NameError:
    print("Advertencia: Variables 'tfidf_vectorizer', 'clean_and_lemmatize', o 'dataset' no encontradas.")
    print("Por favor, ejecuta las celdas correspondientes (Paso 0 y Sub-etapa 1.2) para definirlas.")
    # En un escenario real, aquí podrías detener la ejecución o recargar/redefinir las variables.


# --- Seleccionar algunas reseñas de muestra ---
# Tomamos las mismas 10 reseñas que usamos para ajustar el vectorizador
sample_texts_raw_for_tfidf_viz = [example.numpy().decode('utf-8') for example, label in dataset.take(10)]
sample_texts_processed_for_tfidf_viz = [clean_and_lemmatize(text) for text in sample_texts_raw_for_tfidf_viz]

# --- Transformar las reseñas de muestra usando el vectorizador TF-IDF ajustado ---
# Usamos transform, no fit_transform, porque el vectorizador ya fue ajustado
tfidf_matrix_sample = tfidf_vectorizer.transform(sample_texts_processed_for_tfidf_viz)

# --- Obtener los nombres de las características (palabras) ---
feature_names = tfidf_vectorizer.get_feature_names_out()

# --- Visualizar los términos con TF-IDF más alto para cada reseña de muestra en tabla y gráfico ---

print("\nTérminos con mayor peso TF-IDF para algunas reseñas de muestra:")

# Seleccionar un número limitado de reseñas para mostrar
num_reviews_to_show = 5
num_top_terms = 10 # Mostrar los 10 términos con mayor TF-IDF por reseña

# Aplicar un estilo oscuro a los gráficos
plt.style.use('dark_background')

for i in range(min(num_reviews_to_show, tfidf_matrix_sample.shape[0])):
    print(f"\n--- Reseña de Muestra {i+1} ---")
    # Obtener el vector TF-IDF para la reseña actual
    review_tfidf_vector = tfidf_matrix_sample[i]

    # Convertir el vector disperso a un array denso para facilitar la indexación
    review_tfidf_array = review_tfidf_vector.toarray().flatten()

    # Obtener los índices de los términos con mayor peso TF-IDF
    # Usamos argpartition para eficiencia, seguido de argsort para ordenar
    top_term_indices = np.argpartition(review_tfidf_array, -num_top_terms)[-num_top_terms:]
    # Ordenar los índices por el valor de TF-IDF descendente
    top_term_indices = top_term_indices[np.argsort(-review_tfidf_array[top_term_indices])]

    # Crear una lista de diccionarios para el DataFrame
    top_terms_data = []
    for index in top_term_indices:
        term = feature_names[index]
        tfidf_score = review_tfidf_array[index]
        top_terms_data.append({'Término': term, 'Peso TF-IDF': tfidf_score})

    # Crear y mostrar el DataFrame
    df_top_terms = pd.DataFrame(top_terms_data)
    display(df_top_terms)

    # Crear y mostrar el gráfico de barras
    plt.figure(figsize=(10, 5))
    plt.bar(df_top_terms['Término'], df_top_terms['Peso TF-IDF'])
    plt.ylabel('Peso TF-IDF')
    plt.title(f'Top {num_top_terms} Términos con mayor TF-IDF en Reseña {i+1}')
    plt.xticks(rotation=45, ha='right') # Rotar etiquetas para mejor lectura
    plt.tight_layout()
    plt.show()


# Recomendación:
# Nota cómo las palabras con TF-IDF más alto tienden a ser palabras clave
# que describen el contenido específico o el sentimiento de esa reseña,
# y no palabras comunes como "the", "is", "and" (porque las eliminamos
# y porque IDF las penaliza).

### Sub-etapa 1.4: Exploración de Temas y Extracción de Palabras Clave

Antes de la vectorización final y el modelado de sentimiento, vamos a explorar los temas y términos más importantes en el dataset de reseñas de IMDb utilizando LDA y técnicas de extracción de palabras clave.

### Sub-etapa 1.4.1: Instalar librerías para Modelado de Temas y Extracción de Palabras Clave

In [ ]:
# Instalar gensim para LDA
!pip install gensim

# Instalar una implementación de RAKE
!pip install rake-nltk

### Sub-etapa 1.4.2: Preparar datos para LDA y RAKE

LDA y RAKE generalmente trabajan con texto tokenizado (listas de palabras) más que con las representaciones TF-IDF directamente. Usaremos el resultado de nuestra limpieza y tokenización.

In [ ]:
# Necesitamos los tokens limpios y lematizados para cada reseña.
# Si ya ejecutaste la celda 34b71f7f (Tokenización y Estadísticas),
# deberías tener 'all_tokens' que es una lista plana de todos los tokens.
# Para LDA y RAKE, necesitamos los tokens POR DOCUMENTO.

# Vamos a re-procesar una muestra del dataset para obtener los tokens por documento.
# NOTA: Para el dataset completo, este paso puede ser intensivo en memoria si no se hace de forma eficiente.

import tensorflow as tf
import tensorflow_datasets as tfds
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import nltk
import re
import string
import numpy as np # Importar numpy
import pandas as pd # Importar pandas

# Asegurar que los recursos de NLTK y stop words estén descargados
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
    nltk.data.find('taggers/averaged_perceptron_tagger')
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Recursos de NLTK o stop words no encontrados. Ejecuta las celdas de descarga de la Sub-etapa 1.1.1 y 1.1.4.")

# Asegurar que el dataset esté cargado
try:
    dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
    print("Dataset 'imdb_reviews' cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
    print("Por favor, ejecuta la celda de carga del dataset 'imdb_reviews' (Paso 0).")
    dataset = None

if dataset is not None:
    # Obtener la lista de stop words en inglés
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        print("Recurso 'stopwords' de NLTK no encontrado.")
        stop_words = set()

    # Inicializar Lemmatizer y POS tagger
    lemmatizer = WordNetLemmatizer()
    def get_wordnet_pos(word):
        try:
            tag = nltk.pos_tag([word])[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
            return tag_dict.get(tag, wordnet.NOUN)
        except LookupError:
             # print("Recurso 'averaged_perceptron_tagger' de NLTK no encontrado.")
             return wordnet.NOUN
        except IndexError:
             return wordnet.NOUN

    # Función de limpieza, tokenización, eliminación de stop words y lematización (retorna lista de tokens)
    def clean_tokenize_lemmatize_stopwords_list(text_input):
        if isinstance(text_input, tf.Tensor):
             if tf.size(text_input) > 0 and text_input.dtype == tf.string:
                 text = text_input.numpy()[0].decode('utf-8', errors='ignore')
             else:
                 return []
        elif isinstance(text_input, bytes):
             text = text_input.decode('utf-8', errors='ignore')
        else:
             text = str(text_input)

        text = text.lower()
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        try:
            tokens = word_tokenize(text)
        except LookupError:
            print("Recurso 'punkt' de NLTK no encontrado.")
            return []

        processed_tokens = []
        for word in tokens:
            if word and word not in stop_words:
                lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
                processed_tokens.append(lemma)

        return processed_tokens # Retorna LISTA de tokens


    # --- Procesar una MUESTRA del dataset para LDA/RAKE (para evitar problemas de memoria/tiempo iniciales) ---
    # Podemos aumentar el tamaño de la muestra si es necesario.
    sample_size_for_topic_modeling = 1000 # Procesar 1000 reseñas para el modelado de temas

    processed_documents_tokens = []
    print(f"\nProcesando una muestra de {sample_size_for_topic_modeling} reseñas para Modelado de Temas y Extracción de Keywords...")

    # Iterar sobre una muestra del dataset
    for text_tensor, _ in dataset.take(sample_size_for_topic_modeling).batch(1):
         tokens = clean_tokenize_lemmatize_stopwords_list(text_tensor)
         if tokens: # Solo agregar si la lista de tokens no está vacía
             processed_documents_tokens.append(tokens)

    print(f"Procesamiento de muestra completo. Número de documentos procesados: {len(processed_documents_tokens)}")

else:
    print("\nNo se pudo procesar el dataset para modelado de temas porque no fue cargado exitosamente.")

### Sub-etapa 1.4.3: Modelado de Temas con LDA (Gensim)

Aplicaremos LDA a nuestra muestra de reseñas procesadas para descubrir los temas ocultos. Esto requiere crear un diccionario y un corpus en formato Bag-of-Words que `gensim` pueda usar.

In [ ]:
from gensim import corpora, models

# Asegurarnos de que processed_documents_tokens esté disponible
try:
    processed_documents_tokens
    print(f"Número de documentos procesados para LDA: {len(processed_documents_tokens)}")
except NameError:
    print("La variable 'processed_documents_tokens' no fue encontrada.")
    print("Por favor, ejecuta la celda de preparación de datos para LDA/RAKE (Sub-etapa 1.4.2) primero.")
    processed_documents_tokens = [] # Inicializar como vacío para evitar errores

if processed_documents_tokens:
    # Crear un diccionario a partir de los documentos procesados
    # El diccionario mapea cada palabra única a un ID
    dictionary = corpora.Dictionary(processed_documents_tokens)

    # Opcional: Filtrar palabras raras o muy comunes
    # NoFilter(no_below=5, no_above=0.5, keep_n=100000) # Ejemplo: ignorar palabras que aparecen en <5 docs o >50% docs
    # dictionary.filter_extremes(no_below=5, no_above=0.5)
    # print(f"Tamaño del vocabulario después de filtrar: {len(dictionary)}")


    # Crear el corpus en formato Bag-of-Words (BoW)
    # Para cada documento, crea una lista de tuplas (word_id, word_frequency)
    corpus_bow = [dictionary.doc2bow(doc) for doc in processed_documents_tokens]

    print(f"\nCorpus BoW creado con {len(corpus_bow)} documentos.")

    # --- Entrenar el modelo LDA ---
    # Especificar el número de temas
    num_topics = 5 # Podemos ajustar este número

    print(f"\nEntrenando modelo LDA con {num_topics} temas...")

    # Entrenar el modelo LDA
    lda_model = models.LdaMulticore(corpus_bow,
                                   num_topics=num_topics,
                                   id2word=dictionary,
                                   passes=10,     # Número de pasadas por el corpus durante el entrenamiento
                                   workers=2)     # Número de procesos para entrenamiento paralelo (ajustar según CPU)

    print("Entrenamiento LDA completo.")

    # --- Mostrar los temas encontrados ---
    print(f"\nTemas identificados ({num_topics}):")
    # lda_model.print_topics(num_words=10) # Muestra los 10 términos más importantes para cada tema
    for topic_id, topic_words in lda_model.print_topics(num_words=10):
        print(f"Tema #{topic_id + 1}: {topic_words}")


    # Recomendación:
    # - El número de temas (num_topics) es un hiperparámetro. A menudo se prueba con diferentes valores
    #   y se evalúa la coherencia de los temas o se usa métricas como la coherencia del modelo.
    # - Los temas son distribuciones de palabras. Las palabras con mayor probabilidad en un tema
    #   son las que definen ese tema.
    # - Interpretar los temas requiere mirar las palabras principales y darles un nombre coherente.

else:
     print("\nNo se pudo entrenar LDA porque no hay documentos procesados disponibles.")

### Reinstalando librerías para resolver conflicto de versiones

In [ ]:
# Desinstalar versiones potencialmente conflictivas (opcional pero a veces ayuda)
#!pip uninstall -y gensim numpy scipy

# Reinstalar gensim y sus dependencias
# Pip debería encontrar versiones compatibles con el entorno
#!pip install gensim numpy scipy

**NOTA:** Después de ejecutar esta celda, **reinicia el entorno de ejecución** nuevamente (`Runtime -> Restart runtime`). Es crucial reiniciar después de reinstalar librerías para que los cambios surtan efecto y se carguen las versiones correctas sin conflictos.

Luego, ejecuta las celdas en orden desde el principio (Paso 0), incluyendo las descargas de NLTK, y finalmente la celda del Modelo LDA (`672c0d09`).

### Sub-etapa 1.4.4: Visualizar Temas de LDA con Word Clouds

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd # Asegurarse de que pandas esté importado

# Asegurarse de que lda_model y dictionary estén disponibles
try:
    lda_model
    dictionary
except NameError:
    print("Advertencia: Las variables 'lda_model' o 'dictionary' no fueron encontradas.")
    print("Por favor, ejecuta la celda del modelo LDA (Sub-etapa 1.4.3) y las anteriores (Sub-etapa 1.4.2) para definirlas.")
    # Si las variables no existen, no podemos generar las word clouds


if 'lda_model' in globals() and 'dictionary' in globals():
    print("Generando Word Clouds para cada tema de LDA...")

    # Obtener los temas como listas de palabras y sus pesos
    # lda_model.show_topics() devuelve una lista de tuplas (topic_id, topic_words)
    # topic_words es un string como '"word1" * prob1 + "word2" * prob2 + ...'
    all_topics = lda_model.show_topics(num_topics=lda_model.num_topics, num_words=20, formatted=False)

    # Aplicar el estilo oscuro si ya fue configurado (en la celda 1589fde2)
    # Si no, puedes descomentar la siguiente línea:
    # plt.style.use('dark_background')


    plt.figure(figsize=(15, 8)) # Ajustar tamaño general de la figura

    # Generar una Word Cloud para cada tema
    for i, topic in all_topics:
        # Convertir la lista de tuplas (word, probability) a un diccionario para WordCloud
        topic_dict = dict(topic)

        # Crear la Word Cloud
        # Usamos background_color='black' explícitamente si el estilo oscuro no se aplica globalmente
        wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='viridis').generate_from_frequencies(topic_dict)

        # Mostrar la Word Cloud
        plt.subplot(2, (lda_model.num_topics + 1) // 2, i + 1) # Organizar en filas/columnas
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Tema #{i+1}')

    plt.tight_layout() # Ajustar el layout para evitar solapamiento
    plt.show()

    print("\nWord Clouds de temas LDA generadas.")

else:
    print("\nNo se pudieron generar las Word Clouds. Asegúrate de que el modelo LDA se entrenó correctamente.")

### Sub-etapa 1.4.4: Modelado de Temas con LDA - Iteración 1 (con Filtrado de Vocabulario)

En esta iteración, vamos a mejorar los resultados del modelado de temas aplicando un filtrado al diccionario para eliminar palabras excesivamente frecuentes o raras que pueden dominar los temas. También estableceremos un `seed` para reproducibilidad.

In [ ]:
# from gensim import corpora, models # No usaremos gensim en esta iteración
from sklearn.feature_extraction.text import CountVectorizer # Para Bag-of-Words en scikit-learn
from sklearn.decomposition import LatentDirichletAllocation # Implementación de LDA en scikit-learn
import numpy as np # Necesario para np.array
import pandas as pd # Para mostrar resultados en tabla

# Necesitamos los tokens limpios y lematizados por documento.
# Asegurarnos de que processed_documents_tokens esté disponible (de la celda 55dc36db)
try:
    processed_documents_tokens
    print(f"Número de documentos procesados para LDA: {len(processed_documents_tokens)}")
except NameError:
    print("La variable 'processed_documents_tokens' no fue encontrada.")
    print("Por favor, ejecuta la celda de preparación de datos para LDA/RAKE (Sub-etapa 1.4.2) primero.")
    processed_documents_tokens = [] # Inicializar como vacío para evitar errores


if processed_documents_tokens:
    # --- Preparar datos para scikit-learn LDA ---
    # scikit-learn LDA requiere una matriz de conteos (Bag-of-Words)
    # Necesitamos unir los tokens de cada documento en un string para CountVectorizer
    processed_documents_strings = [" ".join(doc) for doc in processed_documents_tokens]

    # Inicializar y ajustar el CountVectorizer
    # Podemos aplicar filtrado similar al de gensim aquí
    # min_df: Mínima frecuencia de documento (análogo a no_below)
    # max_df: Máxima frecuencia de documento (análogo a no_above)
    # max_features: Limitar el vocabulario (análogo a keep_n)
    count_vectorizer = CountVectorizer(min_df=5, max_df=0.5, max_features=100000)

    # Ajustar el vectorizador al corpus y transformar los documentos en una matriz de conteos
    corpus_counts = count_vectorizer.fit_transform(processed_documents_strings)

    # Obtener los nombres de las características (palabras/tokens)
    feature_names_counts = count_vectorizer.get_feature_names_out()

    print(f"\nCorpus de conteos creado con forma: {corpus_counts.shape}")
    print(f"Tamaño del vocabulario después de CountVectorizer: {len(feature_names_counts)}")


    # --- Entrenar el modelo LDA con scikit-learn ---
    num_topics_sklearn = 5 # Mismo número de temas
    lda_random_state = 42 # Establecer random_state para reproducibilidad en scikit-learn

    print(f"\nEntrenando modelo LDA (scikit-learn) con {num_topics_sklearn} temas y random_state={lda_random_state}...")

    # Inicializar el modelo LDA de scikit-learn
    # n_components: Número de temas
    # random_state: Para reproducibilidad (¡funciona en scikit-learn!)
    # learning_method='batch' o 'online': Batch es más preciso para datasets pequeños/medianos
    # max_iter: Número de iteraciones
    lda_model_sklearn = LatentDirichletAllocation(n_components=num_topics_sklearn,
                                                  random_state=lda_random_state,
                                                  learning_method='batch',
                                                  max_iter=20) # Reducir iteraciones para velocidad inicial

    # Entrenar el modelo LDA
    # Esto puede tardar un poco
    lda_model_sklearn.fit(corpus_counts)

    print("Entrenamiento LDA (scikit-learn) completo.")

    # --- Mostrar los temas encontrados con el modelo scikit-learn ---
    print(f"\nTemas identificados ({num_topics_sklearn}) con scikit-learn LDA:")

    # La salida de scikit-learn LDA es una matriz components_ (temas x palabras)
    # donde cada fila es una distribución de palabras para un tema.
    # Los valores son proporciones logarítmicas. Exponenciarlas da las probabilidades.

    # Función para mostrar los principales términos por tema
    def display_topics_sklearn(model, feature_names, no_top_words):
        for topic_idx, topic in enumerate(model.components_):
            print(f"Tema #{topic_idx + 1}:")
            # Obtener los índices de las palabras más probables y sus nombres
            top_words_indices = topic.argsort()[:-no_top_words - 1:-1]
            top_words = [feature_names[i] for i in top_words_indices]
            # Opcional: mostrar probabilidades (requiere exponenciar y normalizar)
            # top_probs = np.exp(topic[top_words_indices]) / np.sum(np.exp(topic))
            # print(" ".join([f"{word} ({prob:.2f})" for word, prob in zip(top_words, top_probs)]))
            print(" ".join(top_words))
            print("-" * 20)

    display_topics_sklearn(lda_model_sklearn, feature_names_counts, 10) # Mostrar top 10 palabras por tema


    # Recomendación:
    # - Compara estos temas con los de gensim. Deberían ser más estables y quizás más interpretables con el filtrado.
    # - Puedes ajustar min_df, max_df en CountVectorizer y n_components, max_iter en LatentDirichletAllocation.

else:
     print("\nNo se pudo entrenar LDA porque no hay documentos procesados disponibles.")

### Sub-etapa 1.4.5: Visualizar Temas de LDA (Filtrado) con Word Clouds

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd # Asegurarse de que pandas esté importado
import numpy as np # Necesario para manejar arrays

# Usaremos el modelo y diccionario/vectorizador de scikit-learn de la celda anterior (e9834bf1)
try:
    lda_model_sklearn # El modelo LDA de scikit-learn
    feature_names_counts # Los nombres de las características del CountVectorizer
except NameError:
    print("Advertencia: Las variables 'lda_model_sklearn' o 'feature_names_counts' no fueron encontradas.")
    print("Por favor, ejecuta la celda de Modelado de Temas LDA - Iteración 1 (Sub-etapa 1.4.4, celda e9834bf1) primero.")
    # Si las variables no existen, no podemos generar las word clouds


if 'lda_model_sklearn' in globals() and 'feature_names_counts' in globals():
    print("Generando Word Clouds para cada tema del modelo LDA de scikit-learn...")

    # Obtener las distribuciones de palabras por tema desde el modelo scikit-learn
    # model.components_ es una matriz (num_topics, num_features) con las proporciones logarítmicas
    topic_word_distributions = lda_model_sklearn.components_

    # Aplicar el estilo oscuro si ya fue configurado
    plt.style.use('dark_background')

    # Determinar el número de temas
    num_topics_viz = topic_word_distributions.shape[0]

    plt.figure(figsize=(15, 8)) # Ajustar tamaño general de la figura

    # Generar una Word Cloud para cada tema
    # Iteramos sobre cada fila en topic_word_distributions
    for i, topic_dist in enumerate(topic_word_distributions):
        # Obtener los índices de las palabras más importantes para este tema
        # Usamos argsort y luego revertimos para obtener los índices de mayor a menor probabilidad
        # Podemos tomar un número limitado de palabras para la Word Cloud
        num_top_words_wc = 50 # Número de palabras a incluir en la Word Cloud

        # Obtener los índices de las palabras principales (de mayor probabilidad logarítmica)
        top_word_indices = topic_dist.argsort()[:-num_top_words_wc - 1:-1]

        # Crear un diccionario de palabras y sus "pesos" para WordCloud
        # Podemos usar la probabilidad (exp(log_prob)) o simplemente la probabilidad logarítmica
        # WordCloud trabaja con frecuencias o pesos, así que usaremos las probabilidades
        topic_dict = {}
        # Exponenciar las probabilidades logarítmicas y normalizar (opcional, WordCloud puede trabajar con valores no normalizados)
        # La suma de exp(log_prob) para un tema NO suma 1 directamente, es la suma de las *probabilidades* lo que suma 1.
        # Para WordCloud, simplemente usar los valores (quizás exponenciados) funciona bien.
        # Vamos a usar los valores de topic_dist directamente o sus exponenciales. Exp es más representativo de probabilidad.
        # Asegurarse de que los valores sean positivos, lo cual exp(log_prob) garantiza.
        # Normalizar solo para este tema para que sumen 1 para WordCloud
        # No es estrictamente necesario que sumen 1, WordCloud escala por sí solo.
        # Simplemente usaremos los valores exponenciados de las top palabras.
        top_word_probs = np.exp(topic_dist[top_word_indices])
        # Normalizar estas top N probabilidades para que sumen 1 para WordCloud (opcional, pero común)
        top_word_probs_normalized = top_word_probs / np.sum(top_word_probs)

        for j, word_index in enumerate(top_word_indices):
            word = feature_names_counts[word_index]
            # Usamos la probabilidad normalizada para el tamaño en la Word Cloud
            topic_dict[word] = top_word_probs_normalized[j]


        # Crear la Word Cloud (fondo oscuro)
        wordcloud = WordCloud(width=800, height=400, background_color='black', colormap='viridis').generate_from_frequencies(topic_dict)

        # Mostrar la Word Cloud
        plt.subplot(2, (num_topics_viz + 1) // 2, i + 1) # Organizar en filas/columnas
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'Tema #{i+1 + 1} (scikit-learn)') # +1 para índice base 1, +1 para diferenciar iteración

    plt.tight_layout() # Ajustar el layout
    plt.show()

    print("\nWord Clouds de temas LDA (scikit-learn, filtrado) generadas.")

else:
    print("\nNo se pudieron generar las Word Clouds. Asegúrate de que el modelo LDA de scikit-learn se entrenó correctamente.")

### Sub-etapa 1.4.5: Extracción de Palabras Clave (TF-IDF y RAKE)

In [ ]:
# Necesitamos la matriz TF-IDF y los nombres de las características (palabras)
# Si la variable tfidf_vectorizer no está definida (por ejemplo, si reiniciaste
# el entorno sin ejecutar la celda 4704de53), necesitarás ejecutarla primero.
# También necesitamos los documentos procesados (lista de listas de tokens)
# Si 'processed_documents_tokens' no está definido, ejecuta la celda 55dc36db.

# Asegurarnos de que las variables necesarias estén disponibles
try:
    tfidf_vectorizer
    processed_documents_tokens
    # feature_names is obtained from tfidf_vectorizer.get_feature_names_out()
    feature_names = tfidf_vectorizer.get_feature_names_out()
    print(f"Variables 'tfidf_vectorizer', 'processed_documents_tokens' y 'feature_names' encontradas.")
except NameError:
    print("Advertencia: Variables necesarias para Extracción de Palabras Clave no encontradas.")
    print("Por favor, ejecuta las celdas correspondientes (Sub-etapa 1.2 y Sub-etapa 1.4.2) para definirlas.")
    # Inicializar variables para evitar errores si no se encuentran
    tfidf_vectorizer = None
    processed_documents_tokens = []
    feature_names = []


# --- Extracción de Palabras Clave basada en TF-IDF ---
if tfidf_vectorizer is not None and processed_documents_tokens:
    print("\n--- Extracción de Palabras Clave basada en TF-IDF ---")

    # Re-calcular la matriz TF-IDF para la muestra procesada si es necesario
    # Usamos transform si el vectorizador ya fue ajustado en una muestra similar
    # o fit_transform si queremos ajustar en esta muestra específica.
    # Para consistencia con la visualización anterior, usamos el vectorizador ajustado en 10 documentos.
    # Si quieres usar un vectorizador ajustado en una muestra mayor (como los 1000 de LDA),
    # necesitarías reajustar TfidfVectorizer en 'processed_documents_tokens'

    # Aquí usaremos el vectorizador ya ajustado (de la celda 4704de53) en una muestra pequeña
    # Para aplicar a 'processed_documents_tokens' (1000 docs), necesitamos un vectorizador ajustado en ellos.
    # Vamos a reajustar TfidfVectorizer en los 1000 documentos para esta sección.

    from sklearn.feature_extraction.text import TfidfVectorizer

    # Unir los tokens de cada documento en un string para TfidfVectorizer
    processed_documents_strings = [" ".join(doc) for doc in processed_documents_tokens]

    # Inicializar y ajustar el vectorizador TF-IDF en la muestra de 1000 documentos
    tfidf_vectorizer_sample = TfidfVectorizer(max_features=5000) # Usar el mismo límite de features
    tfidf_matrix_sample = tfidf_vectorizer_sample.fit_transform(processed_documents_strings)
    feature_names_sample = tfidf_vectorizer_sample.get_feature_names_out()

    print(f"Vectorizador TF-IDF ajustado en {len(processed_documents_tokens)} documentos.")

    # Mostrar las palabras clave con mayor TF-IDF para algunas reseñas de muestra
    num_reviews_to_show_keywords = 3
    num_top_terms_keywords = 5

    print(f"\nTop {num_top_terms_keywords} Palabras Clave (TF-IDF) para algunas reseñas de muestra:")

    for i in range(min(num_reviews_to_show_keywords, tfidf_matrix_sample.shape[0])):
        review_tfidf_vector = tfidf_matrix_sample[i].toarray().flatten()
        top_term_indices = review_tfidf_vector.argsort()[-num_top_terms_keywords:][::-1] # Indices de mayor a menor

        print(f"  --- Reseña {i+1} ---")
        for index in top_term_indices:
            term = feature_names_sample[index]
            score = review_tfidf_vector[index]
            print(f"    - {term}: {score:.4f}")

else:
    print("\nNo se pudo realizar la Extracción de Palabras Clave basada en TF-IDF. Asegúrate de que las variables necesarias estén definidas.")


# --- Extracción de Palabras Clave con RAKE ---
from rake_nltk import Rake

if processed_documents_tokens:
    print("\n--- Extracción de Palabras Clave con RAKE ---")

    # RAKE trabaja mejor en el texto original o menos procesado, pero podemos probar con el texto lematizado.
    # También necesita stop words. Podemos usar la lista de NLTK.
    try:
        # Asegurarse de que stop_words esté definido (de la celda 98b2a2e1 o 34b71f7f)
        stop_words
    except NameError:
        print("Advertencia: La variable 'stop_words' no fue encontrada. Intentando descargar stop words de NLTK.")
        import nltk
        try:
            nltk.data.find('corpora/stopwords')
        except LookupError:
            nltk.download('stopwords')
        from nltk.corpus import stopwords
        stop_words = set(stopwords.words('english'))


    # Inicializar RAKE con la lista de stop words en inglés
    # RAKE funciona en strings, no en listas de tokens. Usaremos los strings procesados.
    rake = Rake(stopwords=stop_words)

    # Mostrar las palabras clave extraídas por RAKE para algunas reseñas de muestra
    num_reviews_to_show_rake = 3

    print(f"\nTop Palabras/Frases Clave (RAKE) para algunas reseñas de muestra:")

    # Usamos processed_documents_strings que creamos para TF-IDF
    if 'processed_documents_strings' in locals() and processed_documents_strings:
        for i in range(min(num_reviews_to_show_rake, len(processed_documents_strings))):
            review_text = processed_documents_strings[i]
            rake.extract_keywords_from_text(review_text)
            # Obtener las palabras clave clasificadas con sus puntuaciones
            keyword_scores = rake.get_ranked_phrases_with_scores()

            print(f"  --- Reseña {i+1} ---")
            # Mostrar un número limitado de las mejores palabras clave/frases
            for score, phrase in keyword_scores[:5]: # Mostrar las top 5 frases clave
                print(f"    - {phrase}: {score:.4f}")
    else:
        print("No se pudo realizar la Extracción de Palabras Clave con RAKE. Asegúrate de que 'processed_documents_strings' esté disponible.")

else:
    print("\nNo se pudo realizar la Extracción de Palabras Clave con RAKE. Asegúrate de que 'processed_documents_tokens' esté disponible.")

# Recomendación:
# - TF-IDF resalta palabras únicas importantes.
# - RAKE resalta frases clave que capturan conceptos.
# Ambos son útiles para entender el contenido de los documentos.

### Sub-etapa 1.4.6: Visualizar Palabras Clave Extraídas (TF-IDF y RAKE)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Asegurarse de que las variables necesarias de la celda 1.4.5 estén disponibles
try:
    # tfidf_vectorizer_sample is from cell a9335e96
    # tfidf_matrix_sample is from cell a9335e96
    # feature_names_sample is from cell a9335e96
    tfidf_vectorizer_sample
    tfidf_matrix_sample
    feature_names_sample
    # rake is from cell a9335e96
    rake
    # processed_documents_strings is from cell a9335e96
    processed_documents_strings
except NameError:
    print("Advertencia: Variables necesarias para la visualización de Palabras Clave no encontradas.")
    print("Por favor, ejecuta la celda de Extracción de Palabras Clave (Sub-etapa 1.4.5, celda a9335e96) primero.")
    # Inicializar variables para evitar errores si no se encuentran
    tfidf_vectorizer_sample = None
    tfidf_matrix_sample = None
    feature_names_sample = np.array([]) # Inicializar como array vacío de numpy para evitar NameError
    rake = None
    processed_documents_strings = []


# Aplicar el estilo oscuro si ya fue configurado
plt.style.use('dark_background')


print("\n--- Visualización de Palabras Clave Extraídas ---")

# --- Visualización TF-IDF ---
# Corregir la condición para verificar si feature_names_sample no está vacío usando len()
if tfidf_matrix_sample is not None and len(feature_names_sample) > 0:
    print("\nPalabras Clave (TF-IDF) para algunas reseñas de muestra:")

    num_reviews_to_show_viz = 3 # Mostrar visualización para 3 reseñas
    num_top_terms_viz = 10 # Mostrar los 10 términos con mayor TF-IDF

    for i in range(min(num_reviews_to_show_viz, tfidf_matrix_sample.shape[0])):
        print(f"\nReseña de Muestra {i+1} (TF-IDF):")

        # Obtener el vector TF-IDF para la reseña actual
        review_tfidf_vector = tfidf_matrix_sample[i].toarray().flatten()

        # Obtener los índices de los términos con mayor peso TF-IDF y ordenar
        top_term_indices = np.argpartition(review_tfidf_vector, -num_top_terms_viz)[-num_top_terms_viz:]
        top_term_indices = top_term_indices[np.argsort(-review_tfidf_vector[top_term_indices])]

        # Crear DataFrame y Gráfico de Barras
        top_terms_data = []
        for index in top_term_indices:
            term = feature_names_sample[index]
            tfidf_score = review_tfidf_vector[index]
            top_terms_data.append({'Término': term, 'Peso TF-IDF': tfidf_score})

        df_top_terms_viz = pd.DataFrame(top_terms_data)
        display(df_top_terms_viz)

        plt.figure(figsize=(10, 5))
        plt.bar(df_top_terms_viz['Término'], df_top_terms_viz['Peso TF-IDF'])
        plt.ylabel('Peso TF-IDF')
        plt.title(f'Top {num_top_terms_viz} Términos (TF-IDF) - Reseña {i+1}')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()
else:
    print("\nNo se pudo generar la visualización de TF-IDF. Asegúrate de que la Extracción de Palabras Clave (TF-IDF) fue exitosa y generó datos.")


# --- Visualización RAKE ---
# La condición para processed_documents_strings ya debería funcionar si es una lista
if rake is not None and processed_documents_strings:
    print("\nPalabras/Frases Clave (RAKE) para algunas reseñas de muestra:")

    num_reviews_to_show_rake_viz = 3 # Mostrar visualización para 3 reseñas
    num_top_phrases_viz = 5 # Mostrar las top 5 frases RAKE

    for i in range(min(num_reviews_to_show_rake_viz, len(processed_documents_strings))):
        print(f"\nReseña de Muestra {i+1} (RAKE):")

        # Re-extraer keywords para la reseña (ya que RAKE trabaja a nivel de documento individual)
        review_text = processed_documents_strings[i]
        rake.extract_keywords_from_text(review_text)
        keyword_scores = rake.get_ranked_phrases_with_scores()

        # Crear DataFrame para las top frases clave
        top_phrases_data = [{'Frase Clave': phrase, 'Puntuación RAKE': score} for score, phrase in keyword_scores[:num_top_phrases_viz]]
        df_top_phrases_viz = pd.DataFrame(top_phrases_data)
        display(df_top_phrases_viz)

        # NOTA: Un gráfico de barras para RAKE es un poco menos estándar para múltiples frases de longitud variable,
        # pero podríamos hacerlo si fuera necesario, mostrando las frases en el eje X.
        # Por ahora, la tabla muestra claramente las frases y sus puntuaciones.


else:
    print("\nNo se pudo generar la visualización de RAKE. Asegúrate de que la Extracción de Palabras Clave (RAKE) fue exitosa y generó datos.")

# Recomendación:
# Compara los resultados de TF-IDF (palabras individuales importantes) con los de RAKE (frases clave).
# Ambos métodos identifican "palabras clave", pero RAKE es mejor para capturar conceptos multi-palabra.

### Sub-etapa 1.5: Vectorización TF-IDF en el Dataset Completo

Aplicaremos el proceso de limpieza y lematización desarrollado a todo el dataset `imdb_reviews` y luego utilizaremos `TfidfVectorizer` para obtener la representación TF-IDF de cada reseña.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import nltk
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd # Para mostrar información si es necesario
import numpy as np # Necesario para np.array

# Asegurar que los recursos de NLTK y stop words estén descargados
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
    nltk.data.find('taggers/averaged_perceptron_tagger')
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Recursos de NLTK o stop words no encontrados. Ejecuta las celdas de descarga de la Sub-etapa 1.1.1 y 1.1.4.")

# Asegurar que el dataset esté cargado
try:
    # Si la variable 'dataset' ya existe, la usamos.
    print("Utilizando el dataset 'imdb_reviews' cargado previamente.")
except NameError:
    # Si no, intentamos cargarlo.
    print("Dataset 'imdb_reviews' no encontrado. Intentando cargarlo.")
    try:
        dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
        print("Dataset 'imdb_reviews' cargado exitosamente.")
    except Exception as e:
        print(f"Error al cargar el dataset: {e}")
        print("Por favor, ejecuta la celda de carga del dataset 'imdb_reviews' (Paso 0).")
        dataset = None # Asegurar que dataset es None si la carga falla


if dataset is not None:
    # Obtener la lista de stop words en inglés
    try:
        stop_words = set(stopwords.words('english'))
    except LookupError:
        print("Recurso 'stopwords' de NLTK no encontrado.")
        stop_words = set() # Usar un set vacío

    # Inicializar Lemmatizer y POS tagger
    lemmatizer = WordNetLemmatizer()
    # get_wordnet_pos function defined previously (Cell 34b71f7f or 98b2a2e1)
    # Redefinir la función get_wordnet_pos para asegurar que esté disponible
    def get_wordnet_pos(word):
        try:
            tag = nltk.pos_tag([word])[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
            return tag_dict.get(tag, wordnet.NOUN)
        except LookupError:
             return wordnet.NOUN
        except IndexError:
             return wordnet.NOUN


    # Función de limpieza, tokenización, eliminación de stop words y lematización
    # Esta función procesará un string y retornará un string con los tokens unidos por espacio
    def clean_and_lemmatize_text(text_input):
        if isinstance(text_input, tf.Tensor):
             if tf.size(text_input) > 0 and text_input.dtype == tf.string:
                 text = text_input.numpy().decode('utf-8', errors='ignore')
             else:
                 return "" # Retornar string vacío si el tensor está vacío o no es string
        elif isinstance(text_input, bytes):
             text = text_input.decode('utf-8', errors='ignore')
        else: # Asumir que ya es un string
             text = str(text_input)

        text = text.lower()
        text = re.sub(r'<br />', ' ', text)
        text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        try:
            tokens = word_tokenize(text)
        except LookupError:
            print("Recurso 'punkt' de NLTK no encontrado.")
            return "" # Retornar string vacío si falta el recurso de tokenización

        processed_tokens = []
        for word in tokens:
            if word and word not in stop_words:
                lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
                processed_tokens.append(lemma)

        return " ".join(processed_tokens) # Retornar un string con los tokens unidos


    # --- Procesar el dataset completo y recolectar los textos procesados ---
    # Iterar sobre el dataset y aplicar la función de procesamiento.
    # Recolectamos los resultados en una lista de strings.
    # NOTA: Esto puede consumir mucha memoria para datasets muy grandes.
    # Para datasets extremadamente grandes, considera procesar en batches y guardar
    # los resultados en disco o usar tf.data.Dataset con tf.py_function y luego TextVectorization.

    processed_texts_list = []
    labels_list = []
    total_reviews_processed = 0

    print("Procesando dataset completo para TF-IDF (esto puede tardar)...")

    # Usar as_numpy_iterator() para iterar más eficientemente sobre lotes convertidos a numpy
    batch_size_for_processing = 100 # Procesar 100 reseñas a la vez

    for text_batch, label_batch in dataset.batch(batch_size_for_processing).as_numpy_iterator():
        for i in range(len(text_batch)):
            processed_text = clean_and_lemmatize_text(text_batch[i])
            processed_texts_list.append(processed_text)
            labels_list.append(label_batch[i]) # Guardar también las etiquetas

        total_reviews_processed += len(text_batch)
        if total_reviews_processed % 5000 == 0:
            print(f"Procesadas {total_reviews_processed} reseñas...")

    print(f"\nProcesamiento de texto completo. Total de reseñas procesadas: {total_reviews_processed}")
    print(f"Número de textos procesados recolectados: {len(processed_texts_list)}")


    # --- Aplicar TF-IDF al corpus completo de textos procesados ---
    # Inicializar el vectorizador TF-IDF
    # Es crucial limitar max_features para manejar la dimensionalidad y memoria
    # Puedes ajustar este número
    tfidf_max_features = 10000 # Considerar las 10,000 palabras/lemas más importantes

    print(f"\nAplicando TF-IDF con max_features={tfidf_max_features}...")

    tfidf_vectorizer_full = TfidfVectorizer(max_features=tfidf_max_features)

    # Ajustar y transformar el corpus completo
    tfidf_matrix_full = tfidf_vectorizer_full.fit_transform(processed_texts_list)

    # Obtener los nombres de las características (palabras/lemas)
    feature_names_full = tfidf_vectorizer_full.get_feature_names_out()

    print("Vectorización TF-IDF completa.")
    print(f"Forma de la matriz TF-IDF completa: {tfidf_matrix_full.shape}")
    print(f"Número de características (vocabulario TF-IDF): {len(feature_names_full)}")

    # Convertir la lista de etiquetas a un array numpy para consistencia
    labels_array = np.array(labels_list)

    print(f"Forma del array de etiquetas: {labels_array.shape}")

    # Recomendación:
    # La matriz tfidf_matrix_full es una matriz dispersa (sparse matrix) para ahorrar memoria.
    # La mayoría de los modelos de ML (como los de scikit-learn) pueden trabajar directamente con matrices dispersas.
    # Ahora tienes tus datos (tfidf_matrix_full) y etiquetas (labels_array) listos para el entrenamiento del modelo (Paso 2).

else:
    print("\nNo se pudo vectorizar el dataset completo porque no fue cargado exitosamente.")

## Paso 2: Entrenar un modelo de clasificación de sentimiento

Entrenaremos un modelo de Regresión Logística para clasificar reseñas como positivas (1) o negativas (0) utilizando las representaciones TF-IDF y las etiquetas preparadas en el Paso 1.

### Sub-etapa 2.1: División de datos en conjuntos de entrenamiento y validación

In [ ]:
from sklearn.model_selection import train_test_split

# Asegurarnos de que tfidf_matrix_full y labels_array estén disponibles
try:
    tfidf_matrix_full
    labels_array
    print(f"Forma de la matriz TF-IDF completa: {tfidf_matrix_full.shape}")
    print(f"Forma del array de etiquetas: {labels_array.shape}")
except NameError:
    print("Advertencia: Las variables 'tfidf_matrix_full' o 'labels_array' no fueron encontradas.")
    print("Por favor, ejecuta la celda de Vectorización TF-IDF en el Dataset Completo (Sub-etapa 1.5) primero.")
    # Salir o manejar el error si los datos no están disponibles


if 'tfidf_matrix_full' in globals() and 'labels_array' in globals():
    # Dividir los datos. Usaremos un 80% para entrenamiento y 20% para validación.
    # stratify=labels_array asegura que la proporción de clases (0s y 1s) sea similar en ambos conjuntos.
    # random_state=42 asegura que la división sea la misma cada vez que ejecutas el código.
    X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix_full,
                                                        labels_array,
                                                        test_size=0.2,
                                                        random_state=42,
                                                        stratify=labels_array)

    print("\nDatos divididos en conjuntos de entrenamiento y validación:")
    print(f"Forma de X_train: {X_train.shape}")
    print(f"Forma de X_test: {X_test.shape}")
    print(f"Forma de y_train: {y_train.shape}")
    print(f"Forma de y_test: {y_test.shape}")

    # Opcional: Verificar la distribución de clases en los conjuntos
    # print("\nDistribución de clases en y_train:")
    # display(pd.Series(y_train).value_counts(normalize=True))
    # print("\nDistribución de clases en y_test:")
    # display(pd.Series(y_test).value_counts(normalize=True))

else:
    print("\nNo se pudo dividir los datos. Asegúrate de que los datos TF-IDF y etiquetas estén cargados.")

### Sub-etapa 2.2: Entrenamiento del modelo de Regresión Logística

In [ ]:
from sklearn.linear_model import LogisticRegression
# Importar métricas para evaluación (aunque las usaremos en la siguiente celda)
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Asegurarnos de que los datos de entrenamiento estén disponibles
try:
    X_train, y_train
    print(f"Usando datos de entrenamiento con forma X_train: {X_train.shape}, y_train: {y_train.shape}")
except NameError:
    print("Advertencia: Los datos de entrenamiento (X_train, y_train) no fueron encontrados.")
    print("Por favor, ejecuta la celda de División de datos (Sub-etapa 2.1) primero.")


if 'X_train' in globals() and 'y_train' in globals():
    print("\nInicializando y entrenando el modelo de Regresión Logística...")

    # Inicializar el modelo de Regresión Logística
    # class_weight='balanced' ayuda a manejar posibles desbalances en las clases
    # solver='liblinear' es bueno para datasets pequeños y medianos con L1/L2 regularization
    # random_state=42 para reproducibilidad
    model = LogisticRegression(solver='liblinear', random_state=42, class_weight='balanced', max_iter=1000)


    # Entrenar el modelo
    # Esto puede tardar un poco dependiendo del tamaño del dataset y max_features
    model.fit(X_train, y_train)

    print("Entrenamiento del modelo de Regresión Logística completo.")
    print(f"El modelo ha aprendido {model.n_features_in_} características (palabras/lemas).")

else:
    print("\nNo se pudo entrenar el modelo. Asegúrate de que los datos de entrenamiento estén disponibles.")

### Sub-etapa 2.3: Evaluación del modelo

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns # Para visualizar la matriz de confusión

# Asegurarnos de que el modelo entrenado y los datos de prueba estén disponibles
try:
    model # El modelo entrenado
    X_test, y_test # Datos de validación/prueba
    print("Modelo entrenado y datos de prueba encontrados.")
except NameError:
    print("Advertencia: El modelo entrenado o los datos de prueba (X_test, y_test) no fueron encontrados.")
    print("Por favor, ejecuta las celdas de Entrenamiento del modelo (Sub-etapa 2.2) y División de datos (Sub-etapa 2.1) primero.")


if 'model' in globals() and 'X_test' in globals() and 'y_test' in globals():
    print("\nEvaluando el modelo en el conjunto de validación...")

    # Realizar predicciones en el conjunto de validación
    y_pred = model.predict(X_test)

    # --- Mostrar Métricas de Clasificación ---
    print("\nReporte de Clasificación:")
    # target_names permite especificar los nombres de las clases (0 -> negativo, 1 -> positivo)
    print(classification_report(y_test, y_pred, target_names=['Negativo', 'Positivo']))

    # --- Mostrar Matriz de Confusión ---
    print("\nMatriz de Confusión:")
    cm = confusion_matrix(y_test, y_pred)

    # Visualizar la matriz de confusión
    plt.figure(figsize=(6, 5))
    # Aplicar el estilo oscuro si ya fue configurado
    plt.style.use('dark_background') # Asegurarse de que el estilo esté activo
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo', 'Positivo'], yticklabels=['Negativo', 'Positivo'])
    plt.xlabel('Predicción')
    plt.ylabel('Valor Real')
    plt.title('Matriz de Confusión')
    plt.show()

    # Mostrar Accuracy (opcional, ya está en el reporte pero es una métrica clave)
    # accuracy = accuracy_score(y_test, y_pred)
    # print(f"\nAccuracy: {accuracy:.4f}")

    # Recomendación:
    # - Precision: De todas las reseñas que el modelo predijo como Positivas, ¿cuántas eran realmente Positivas?
    # - Recall: De todas las reseñas que eran realmente Positivas, ¿cuántas predijo el modelo correctamente?
    # - F1-score: Es la media armónica de Precision y Recall. Una buena métrica única cuando Precision y Recall son importantes.
    # - Support: Número de instancias reales de cada clase en el conjunto de prueba.
    # - Matriz de Confusión: Muestra cuántas predicciones fueron Verdaderos Positivos (VP), Verdaderos Negativos (VN), Falsos Positivos (FP), y Falsos Negativos (FN).

else:
    print("\nNo se pudo evaluar el modelo. Asegúrate de que el modelo y los datos de prueba estén disponibles.")

### Sub-etapa 2.4: Guardar el modelo entrenado

In [ ]:
import pickle
import os

# Asegurarnos de que el modelo entrenado esté disponible
try:
    model
    print("Modelo entrenado encontrado.")
except NameError:
    print("Advertencia: El modelo entrenado no fue encontrado.")
    print("Por favor, ejecuta la celda de Entrenamiento del modelo (Sub-etapa 2.2) primero.")


if 'model' in globals():
    # Definir el nombre del archivo para guardar el modelo
    model_filename = 'logistic_regression_sentiment_model.pkl'

    # Guardar el modelo en un archivo .pkl
    with open(model_filename, 'wb') as f:
        pickle.dump(model, f)

    print(f"\nModelo entrenado guardado como '{model_filename}'")
    print(f"Tamaño del archivo del modelo: {os.path.getsize(model_filename)} bytes")

    # Recomendación:
    # Este archivo .pkl contiene el modelo entrenado. Puedes cargarlo en otro script
    # o notebook para hacer predicciones en datos nuevos sin tener que reentrenar.
    # Por ejemplo: loaded_model = pickle.load(open('logistic_regression_sentiment_model.pkl', 'rb'))
    # Para usar este modelo en el dataset de metadatos, necesitarás aplicar la misma
    # vectorización TF-IDF (usando el *mismo* tfidf_vectorizer_full que ajustamos en el Paso 1.5)
    # a la información textual de ese dataset antes de pasársela al modelo.

else:
    print("\nNo se pudo guardar el modelo. Asegúrate de que el modelo esté disponible.")

## Paso 3: Adaptar el conocimiento al dataset `df_movie_metadata`

Cargaremos el modelo de Regresión Logística entrenado y el vectorizador TF-IDF guardados, y los utilizaremos para predecir el sentimiento (basado en las palabras clave) para cada película en el dataset `df_movie_metadata`.

### Sub-etapa 3.1: Cargar el modelo y el vectorizador guardados

In [ ]:
import pickle
import os

# Definir los nombres de los archivos guardados (de Sub-etapa 2.4 y 1.5)
model_filename = 'logistic_regression_sentiment_model.pkl'
# El vectorizador TF-IDF completo fue guardado implícitamente en la variable tfidf_vectorizer_full
# Necesitamos asegurarnos de que esta variable esté disponible o, idealmente, guardar/cargar también el vectorizador.
# Como no lo guardamos explícitamente en un .pkl, vamos a asumir que la variable
# tfidf_vectorizer_full de la celda 1.5 está disponible. Si no, necesitaríamos re-ejecutar esa celda
# o modificar la celda 1.5 para guardar el vectorizador también.

# Para mayor robustez, vamos a modificar la celda 1.5 para guardar el vectorizador y cargarlo aquí.
# **NOTA:** Necesitarás re-ejecutar la celda 1.5 después de la modificación para guardar el vectorizador.


# --- Cargar el modelo ---
try:
    with open(model_filename, 'rb') as f:
        loaded_model = pickle.load(f)
    print(f"Modelo '{model_filename}' cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo del modelo '{model_filename}' no fue encontrado.")
    loaded_model = None
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    loaded_model = None


# --- Cargar el vectorizador TF-IDF ---
# Asumiendo que tfidf_vectorizer_full está disponible en el entorno.
# Si no, necesitaríamos cargarlo desde un archivo si lo hubiéramos guardado en la Sub-etapa 1.5.
try:
    tfidf_vectorizer_full
    print("Variable 'tfidf_vectorizer_full' encontrada en el entorno.")
    loaded_vectorizer = tfidf_vectorizer_full
except NameError:
    print("Advertencia: La variable 'tfidf_vectorizer_full' no fue encontrada en el entorno.")
    print("Por favor, ejecuta la celda de Vectorización TF-IDF en el Dataset Completo (Sub-etapa 1.5).")
    # Si hubiéramos guardado el vectorizador, lo cargaríamos aquí:
    # vectorizer_filename = 'tfidf_vectorizer_full.pkl'
    # try:
    #     with open(vectorizer_filename, 'rb') as f:
    #         loaded_vectorizer = pickle.load(f)
    #     print(f"Vectorizer '{vectorizer_filename}' cargado exitosamente.")
    # except FileNotFoundError:
    #     print(f"Error: El archivo del vectorizador '{vectorizer_filename}' no fue encontrado.")
    #     loaded_vectorizer = None
    # except Exception as e:
    #     print(f"Error al cargar el vectorizador: {e}")
    #     loaded_vectorizer = None
    loaded_vectorizer = None # Asegurar que es None si no se encuentra


if loaded_model is not None and loaded_vectorizer is not None:
    print("\nModelo y Vectorizador listos para inferencia.")
else:
    print("\nNo se pudieron cargar el Modelo o el Vectorizador. No se puede proceder con la inferencia.")

### Sub-etapa 3.2: Preprocesar y Vectorizar la columna `plot_keywords` de `df_movie_metadata`

Aplicaremos el mismo proceso de limpieza, lematización y eliminación de stop words a la columna `plot_keywords` del dataset `df_movie_metadata`. Luego, usaremos el `tfidf_vectorizer_full` entrenado previamente para convertir estas palabras clave en vectores TF-IDF.

In [ ]:
# Asegurarnos de que df_movie_metadata esté cargado y limpio (sin NaNs)
try:
    # Verificamos si la variable df_movie_metadata existe y tiene la columna 'plot_keywords'
    if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame) and 'plot_keywords' in df_movie_metadata.columns:
         print("Utilizando el DataFrame df_movie_metadata (limpio) cargado previamente.")
         # Asegurarnos de que no tenga NaNs en 'plot_keywords', aunque ya eliminamos filas con NaNs
         # En un escenario real, podrías querer imputar solo esta columna si no eliminaste filas.
         # df_movie_metadata['plot_keywords'].fillna('', inplace=True) # Ejemplo de imputación simple

         # Si el DataFrame fue limpiado eliminando filas, ya no debería haber NaNs.
         if df_movie_metadata['plot_keywords'].isnull().any():
             print("Advertencia: La columna 'plot_keywords' aún contiene valores nulos. Imputando con string vacío.")
             df_movie_metadata['plot_keywords'].fillna('', inplace=True)


    else:
        print("Advertencia: La variable df_movie_metadata no fue encontrada, no es un DataFrame o no tiene la columna 'plot_keywords'.")
        print("Por favor, ejecuta las celdas de carga del dataset Kaggle y manejo de faltantes (Paso 0 y Sub-etapa 1.3).")
        df_movie_metadata = None # Asegurar que es None si no está disponible

except NameError:
    print("Advertencia: La variable df_movie_metadata no fue encontrada.")
    print("Por favor, ejecuta las celdas de carga del dataset Kaggle y manejo de faltantes (Paso 0 y Sub-etapa 1.3).")
    df_movie_metadata = None


# Asegurarnos de que loaded_vectorizer esté disponible
try:
    loaded_vectorizer
    print("Vectorizador TF-IDF cargado encontrado.")
except NameError:
    print("Advertencia: El vectorizador TF-IDF ('loaded_vectorizer') no fue encontrado.")
    print("Por favor, ejecuta la celda de Carga de modelo y vectorizador (Sub-etapa 3.1).")
    loaded_vectorizer = None


if df_movie_metadata is not None and loaded_vectorizer is not None:
    # Necesitamos la misma función de limpieza/lematización/stop words que usamos en el entrenamiento
    # Asegurar que la función clean_and_lemmatize_text esté definida (de la celda 78c02b5d)
    try:
        clean_and_lemmatize_text
    except NameError:
        print("Advertencia: La función 'clean_and_lemmatize_text' no fue encontrada.")
        print("Por favor, ejecuta la celda de Vectorización TF-IDF en el Dataset Completo (Sub-etapa 1.5) para definirla.")
        # Si la función no está definida, no podemos procesar el texto.
        clean_and_lemmatize_text = None # Set to None to indicate it's missing

    if clean_and_lemmatize_text is not None:
        print("\nPreprocesando la columna 'plot_keywords'...")

        # Aplicar la función de preprocesamiento a cada keyword string
        # La columna 'plot_keywords' es una string con palabras separadas por '|'
        # Necesitamos convertir 'palabra1|palabra2' a 'palabra1 palabra2' antes de limpiar
        # y luego aplicar la misma limpieza que usamos para las reseñas.
        # NOTA: Esta limpieza puede ser diferente si el formato de keywords es muy distinto al de reseñas.
        # Asumiremos que la limpieza general es aplicable después de reemplazar '|' por espacio.

        processed_keywords_list = df_movie_metadata['plot_keywords'].apply(
            lambda x: clean_and_lemmatize_text(x.replace('|', ' ')) if isinstance(x, str) else clean_and_lemmatize_text('')
        ).tolist()

        print("Preprocesamiento de 'plot_keywords' completo.")
        print(f"Número de entradas procesadas: {len(processed_keywords_list)}")

        print("\nAplicando vectorizador TF-IDF entrenado a las palabras clave procesadas...")

        # Usar el vectorizador cargado para transformar los textos procesados
        # Usamos transform(), NO fit_transform(), para usar el vocabulario y pesos aprendidos en el entrenamiento
        keywords_tfidf_matrix = loaded_vectorizer.transform(processed_keywords_list)

        print("Vectorización de palabras clave completa.")
        print(f"Forma de la matriz TF-IDF de palabras clave: {keywords_tfidf_matrix.shape}")
        # Nota: La segunda dimensión (número de columnas) debe ser la misma que la del vectorizador entrenado.


    else:
        print("\nNo se pudo preprocesar y vectorizar las palabras clave. Asegúrate de que la función de preprocesamiento esté disponible.")

else:
    print("\nNo se pudo preprocesar y vectorizar las palabras clave. Asegúrate de que df_movie_metadata y el vectorizador estén disponibles.")

### Sub-etapa 3.3: Aplicar el modelo de sentimiento para inferencia

In [ ]:
import numpy as np # Importar numpy si es necesario

# Asegurarnos de que el modelo cargado y la matriz TF-IDF de palabras clave estén disponibles
try:
    loaded_model
    keywords_tfidf_matrix
    print("Modelo cargado y matriz TF-IDF de palabras clave encontrados.")
except NameError:
    print("Advertencia: El modelo cargado ('loaded_model') o la matriz TF-IDF de palabras clave ('keywords_tfidf_matrix') no fueron encontrados.")
    print("Por favor, ejecuta las celdas de Carga de modelo (Sub-etapa 3.1) y Preprocesamiento/Vectorización de palabras clave (Sub-etapa 3.2).")
    loaded_model = None
    keywords_tfidf_matrix = None


if loaded_model is not None and keywords_tfidf_matrix is not None:
    print("\nAplicando el modelo de sentimiento a las representaciones TF-IDF de palabras clave...")

    # --- Predecir las etiquetas de sentimiento (0 o 1) ---
    # predictions = loaded_model.predict(keywords_tfidf_matrix)
    # print("\nPredicciones de sentimiento (etiquetas 0 o 1) generadas.")
    # print(f"Primeras 10 predicciones: {predictions[:10]}")

    # --- Predecir las probabilidades de sentimiento positivo ---
    # Esto es a menudo más útil para entender el "grado" de positividad/negatividad
    # La columna 1 de predict_proba() es la probabilidad de la clase positiva (1)
    sentiment_probabilities = loaded_model.predict_proba(keywords_tfidf_matrix)[:, 1]

    print("\nPuntuaciones de probabilidad de sentimiento positivo generadas.")
    print(f"Primeras 10 probabilidades de sentimiento positivo: {sentiment_probabilities[:10]}")

    # --- Añadir las puntuaciones de sentimiento al DataFrame df_movie_metadata ---
    # Asegurarnos de que df_movie_metadata esté disponible
    try:
        df_movie_metadata
        # Asegurarse de que el número de probabilidades coincide con el número de filas en df_movie_metadata
        if len(sentiment_probabilities) == len(df_movie_metadata):
            df_movie_metadata['predicted_sentiment_score'] = sentiment_probabilities
            print("\nColumna 'predicted_sentiment_score' añadida a df_movie_metadata.")
            # Mostrar las primeras filas con la nueva columna
            display(df_movie_metadata[['movie_title', 'plot_keywords', 'predicted_sentiment_score']].head())

        else:
            print("Advertencia: El número de predicciones no coincide con el número de filas en df_movie_metadata.")
            print("Asegúrate de que el preprocesamiento y la vectorización se aplicaron al DataFrame correcto y completo.")

    except NameError:
        print("Advertencia: La variable df_movie_metadata no fue encontrada.")
        print("No se pudieron añadir las puntuaciones de sentimiento al DataFrame.")


    # Recomendación:
    # La columna 'predicted_sentiment_score' ahora contiene la probabilidad de que,
    # basado en sus plot_keywords, el modelo de sentimiento entrenado lo clasificaría como positivo.
    # Este puntaje puede usarse para ordenar, filtrar o analizar películas.
    # Ten en cuenta que esta es una predicción basada *solo* en las plot_keywords,
    # que es una información limitada comparada con una reseña completa.

else:
    print("\nNo se pudo aplicar el modelo de sentimiento. Asegúrate de que el modelo y los datos vectorizados estén disponibles.")

### Visualización de los Resultados de la Inferencia de Sentimiento

Vamos a visualizar la distribución de las puntuaciones de sentimiento predichas y ver ejemplos de películas con sus puntuaciones.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Asegurarse de que pandas esté importado

# Asegurarnos de que df_movie_metadata esté cargado y contenga la columna 'predicted_sentiment_score'
try:
    if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame) and 'predicted_sentiment_score' in df_movie_metadata.columns:
         print("Utilizando el DataFrame df_movie_metadata con la columna 'predicted_sentiment_score'.")
    else:
        print("Advertencia: La variable df_movie_metadata no fue encontrada, no es un DataFrame o no tiene la columna 'predicted_sentiment_score'.")
        print("Por favor, ejecuta las celdas de carga del dataset Kaggle, manejo de faltantes (Paso 0 y Sub-etapa 1.3) y Aplicación del modelo (Sub-etapa 3.3).")
        df_movie_metadata = None # Asegurar que es None si no está disponible

except NameError:
    print("Advertencia: La variable df_movie_metadata no fue encontrada.")
    print("Por favor, ejecuta las celdas de carga del dataset Kaggle, manejo de faltantes (Paso 0 y Sub-etapa 1.3) y Aplicación del modelo (Sub-etapa 3.3).")
    df_movie_metadata = None


if df_movie_metadata is not None:
    print("\n--- Visualización de Resultados de Inferencia ---")

    # --- Mostrar ejemplos de películas con sus puntuaciones predichas ---
    print("\nEjemplos de películas con sus palabras clave, puntuación IMDb y puntuación de sentimiento predicha:")
    # Seleccionar algunas columnas relevantes y las primeras filas
    display(df_movie_metadata[['movie_title', 'plot_keywords', 'imdb_score', 'predicted_sentiment_score']].head())

    # Opcional: Mostrar películas con las puntuaciones más altas y más bajas
    print("\nPelículas con las puntuaciones de sentimiento predichas más altas:")
    display(df_movie_metadata.sort_values(by='predicted_sentiment_score', ascending=False)[['movie_title', 'imdb_score', 'predicted_sentiment_score']].head())

    print("\nPelículas con las puntuaciones de sentimiento predichas más bajas:")
    display(df_movie_metadata.sort_values(by='predicted_sentiment_score', ascending=True)[['movie_title', 'imdb_score', 'predicted_sentiment_score']].head())


    # --- Histograma de la distribución de las puntuaciones de sentimiento predichas ---
    print("\nDistribución de las puntuaciones de sentimiento predichas:")

    plt.figure(figsize=(8, 5))
    # Aplicar el estilo oscuro
    plt.style.use('dark_background')

    sns.histplot(df_movie_metadata['predicted_sentiment_score'], bins=50, kde=True)
    plt.title('Distribución de las Puntuaciones de Sentimiento Predichas')
    plt.xlabel('Puntuación de Sentimiento Predicha (Probabilidad de Positivo)')
    plt.ylabel('Frecuencia')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    # Opcional: Scatter plot de IMDb Score vs Predicted Sentiment Score
    # print("\nScatter plot de IMDb Score vs Predicted Sentiment Score:")
    # plt.figure(figsize=(8, 5))
    # plt.style.use('dark_background')
    # sns.scatterplot(data=df_movie_metadata, x='imdb_score', y='predicted_sentiment_score', alpha=0.6)
    # plt.title('IMDb Score vs Predicted Sentiment Score (basado en Keywords)')
    # plt.xlabel('IMDb Score Original')
    # plt.ylabel('Puntuación de Sentimiento Predicha (Keywords)')
    # plt.grid(True, linestyle='--', alpha=0.6)
    # plt.show()

    # Recomendación:
    # - La tabla te permite ver ejemplos concretos de cómo se predijo el sentimiento para películas específicas.
    # - El histograma muestra si las predicciones tienden a ser más positivas, negativas o distribuidas uniformemente.
    # - Recuerda que esta predicción se basa *únicamente* en las 'plot_keywords', no en reseñas completas,
    #   por lo que puede no alinearse perfectamente con el IMDb Score general de la película.

else:
    print("\nNo se pudo generar la visualización de los resultados de inferencia. Asegúrate de que df_movie_metadata con la columna de sentimiento predicho esté disponible.")

## Paso 4: Generar representaciones vectoriales para `df_movie_metadata`

Crearemos un vector numérico para cada película en el dataset de metadatos combinando sus características relevantes. Este vector servirá como entrada para la reducción de dimensionalidad y visualización.

### Sub-etapa 4.1: Selección y Preparación de Características

Seleccionaremos las columnas numéricas y categóricas a incluir en la representación vectorial y aplicaremos la codificación necesaria.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Asegurarnos de que df_movie_metadata esté cargado y limpio
try:
    if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame):
         print("Utilizando el DataFrame df_movie_metadata (limpio) cargado previamente.")
         print(f"Columnas disponibles: {df_movie_metadata.columns.tolist()}")
    else:
        print("Advertencia: La variable df_movie_metadata no fue encontrada o no es un DataFrame.")
        print("Por favor, ejecuta las celdas de carga del dataset Kaggle y manejo de faltantes (Paso 0 y Sub-etapa 1.3).")
        df_movie_metadata = None # Asegurar que es None si no está disponible

except NameError:
    print("Advertencia: La variable df_movie_metadata no fue encontrada.")
    print("Por favor, ejecuta las celdas de carga del dataset Kaggle y manejo de faltantes (Paso 0 y Sub-etapa 1.3).")
    df_movie_metadata = None


if df_movie_metadata is not None:
    # --- Seleccionar columnas a incluir ---
    # Columnas numéricas (ya manejamos los NaNs eliminando filas)
    numeric_features = [
        'num_critic_for_reviews', 'duration', 'director_facebook_likes',
        'actor_3_facebook_likes', 'actor_2_facebook_likes', 'actor_1_facebook_likes',
        'gross', 'num_voted_users', 'cast_total_facebook_likes',
        'facenumber_in_poster', 'num_user_for_reviews', 'budget',
        'title_year', 'imdb_score', 'movie_facebook_likes',
        'predicted_sentiment_score' # Nuestra columna de sentimiento predicho
    ]

    # Columnas categóricas (seleccionamos algunas para codificar)
    # NOTA: One-Hot Encoding puede crear muchas columnas si hay muchas categorías únicas.
    # Podríamos necesitar filtrar o usar otras técnicas (ej. Target Encoding, Embeddings categóricos)
    # si las columnas tienen alta cardinalidad.
    categorical_features = ['genres', 'country', 'language', 'content_rating']

    # Asegurarnos de que las columnas seleccionadas existan en el DataFrame
    selected_features = numeric_features + categorical_features
    existing_features = [col for col in selected_features if col in df_movie_metadata.columns]
    missing_features = [col for col in selected_features if col not in df_movie_metadata.columns]

    if missing_features:
        print(f"Advertencia: Las siguientes columnas seleccionadas no se encontraron en el DataFrame: {missing_features}")
        print("Usando solo las columnas existentes.")
        numeric_features = [col for col in numeric_features if col in existing_features]
        categorical_features = [col for col in categorical_features if col in existing_features]
        existing_features = numeric_features + categorical_features # Actualizar la lista

    print(f"\nColumnas numéricas seleccionadas: {numeric_features}")
    print(f"Columnas categóricas seleccionadas: {categorical_features}")


    # --- Preparar transformaciones ---
    # Para columnas numéricas: Escalar (MinMaxScaler es común para poner todo en un rango similar)
    # Para columnas categóricas: One-Hot Encode (convierte categorías a vectores binarios)

    # Crear transformadores para cada tipo de columna
    numeric_transformer = Pipeline(steps=[
        ('scaler', MinMaxScaler())
    ])

    # Handle unknown categories in OneHotEncoder by ignoring them or adding a placeholder
    # Setting handle_unknown='ignore' is often safer for inference on new data
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combinar transformadores usando ColumnTransformer
    # Esto aplicará las transformaciones correctas a las columnas correctas
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough' # Mantener otras columnas si las hay (no aplicable aquí ya que seleccionamos)
    )

    # --- Aplicar las transformaciones al DataFrame ---
    print("\nAplicando preprocesamiento y combinando características...")

    # Ajustar y transformar los datos
    # Esto aplica el escalado a numéricas y one-hot encoding a categóricas
    movie_features_vectorized = preprocessor.fit_transform(df_movie_metadata[existing_features])

    print("Generación de representaciones vectoriales completa.")
    print(f"Forma de la matriz de representaciones vectoriales: {movie_features_vectorized.shape}")
    # El número de columnas en la matriz resultante es la suma de:
    # (número de columnas numéricas) + (suma de categorías únicas en cada columna categórica después de OHE)

    # Recomendación:
    # - La matriz movie_features_vectorized ahora contiene el vector numérico para cada película.
    # - Las columnas originales 'movie_title' y 'movie_imdb_link' no se incluyeron en el vector,
    #   pero son útiles como metadatos para la visualización. Podemos guardarlas por separado
    #   o asegurarnos de que se mantengan alineadas con los vectores.

else:
    print("\nNo se pudieron generar las representaciones vectoriales. Asegúrate de que df_movie_metadata esté cargado y limpio.")

## Paso 5: Aplicar Reducción de Dimensionalidad y Visualización

Reduciremos las representaciones vectoriales de las películas a 2 o 3 dimensiones y prepararemos los archivos para visualizarlas en el TensorFlow Projector.

### Sub-etapa 5.1: Aplicar Reducción de Dimensionalidad (t-SNE)

In [ ]:
from sklearn.manifold import TSNE
import numpy as np

# Asegurarnos de que movie_features_vectorized esté disponible
try:
    movie_features_vectorized
    # Corregir el typo en el mensaje de print
    print(f"Utilizando la matriz de representaciones vectoriales con forma: {movie_features_vectorized.shape}")
except NameError:
    print("Advertencia: La variable 'movie_features_vectorized' no fue encontrada.")
    print("Por favor, ejecuta la celda de Generación de representaciones vectoriales (Sub-etapa 4.1) primero.")
    movie_features_vectorized = None # Asegurar que es None si no está disponible


if movie_features_vectorized is not None:
    print("\nAplicando t-SNE para reducir la dimensionalidad a 2 componentes (esto puede tardar)...")

    # Inicializar el modelo t-SNE
    # n_components: Número de dimensiones de salida (2 para visualización 2D)
    # random_state: Para reproducibilidad
    # perplexity: Parámetro sensible
    # max_iter: Número de iteraciones
    # init="random": Usar inicialización aleatoria para matrices dispersas
    tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=300, init="random")

    # Aplicar t-SNE a los datos
    movie_tsne_2d = tsne.fit_transform(movie_features_vectorized)

    print("Reducción de dimensionalidad con t-SNE (2D) completa.")
    print(f"Forma de la matriz de t-SNE 2D: {movie_tsne_2d.shape}")

    # Si quieres 3D para el proyector:
    print("\nAplicando t-SNE para reducir la dimensionalidad a 3 componentes (esto puede tardar)...")
    # n_components: Número de dimensiones de salida (3 para visualización 3D)
    # random_state: Para reproducibilidad
    # perplexity: Parámetro sensible
    # max_iter: Número de iteraciones
    # init="random": Usar inicialización aleatoria
    tsne_3d = TSNE(n_components=3, random_state=42, perplexity=30, max_iter=300, init="random")
    movie_tsne_3d = tsne_3d.fit_transform(movie_features_vectorized)
    print("Reducción de dimensionalidad con t-SNE (3D) completa.")
    print(f"Forma de la matriz de t-SNE 3D: {movie_tsne_3d.shape}")

    # Decidimos qué representación usaremos para el proyector (ej. 3D)
    movie_embeddings_for_projector = movie_tsne_3d
    print(f"\nSe usará la representación 3D de t-SNE para el Projector: {movie_embeddings_for_projector.shape}")

else:
    print("\nNo se pudo aplicar t-SNE. Asegúrate de que las representaciones vectoriales estén disponibles.")

### Sub-etapa 5.2: Preparar archivos .tsv para TensorFlow Projector

Necesitamos dos archivos:

1.  **`vectors.tsv`:** Contiene las coordenadas 2D o 3D para cada punto (película) después de la reducción de dimensionalidad. Cada fila es un punto, cada columna es una dimensión.
2.  **`metadata.tsv`:** Contiene los metadatos para cada punto (película). Cada fila corresponde a una película en `vectors.tsv`. La primera fila es el encabezado. Incluiremos columnas interesantes como título de la película, género, puntuación IMDb, y nuestra puntuación de sentimiento predicha.

In [ ]:
import os
import numpy as np # Asegurarse de que numpy esté importado
import pandas as pd # Asegurarse de que pandas esté importado

# Asegurarnos de que movie_embeddings_for_projector y df_movie_metadata estén disponibles
try:
    movie_embeddings_for_projector # La matriz de t-SNE 3D o 2D
    if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame):
        print(f"Matriz de embeddings para Projector con forma: {movie_embeddings_for_projector.shape}")
        print("DataFrame df_movie_metadata encontrado.")
    else:
        print("Advertencia: La variable df_movie_metadata no fue encontrada o no es un DataFrame.")
        print("Por favor, ejecuta las celdas de carga del dataset Kaggle y manejo de faltantes (Paso 0 y Sub-etapa 1.3).")
        df_movie_metadata = None # Asegurar que es None si no está disponible

except NameError:
    print("Advertencia: La variable 'movie_embeddings_for_projector' o 'df_movie_metadata' no fueron encontradas.")
    print("Por favor, ejecuta las celdas de Reducción de Dimensionalidad (Sub-etapa 5.1) y carga/limpieza de df_movie_metadata.")
    movie_embeddings_for_projector = None
    df_movie_metadata = None


# --- Definir la ruta de destino en Google Drive ---
# Asegúrate de que Google Drive esté montado (ejecutando la celda del Paso 0)
drive_output_path = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1'

# Crear la carpeta de destino si no existe
if not os.path.exists(drive_output_path):
    try:
        os.makedirs(drive_output_path)
        print(f"\nCarpeta de destino en Drive creada: {drive_output_path}")
    except OSError as e:
        print(f"\nError: No se pudo crear la carpeta en Drive {drive_output_path}. Asegúrate de que Drive esté montado y la ruta sea válida.")
        drive_output_path = None # Anular la ruta si falla la creación

if movie_embeddings_for_projector is not None and df_movie_metadata is not None and drive_output_path is not None:

    # --- Preparar y guardar el archivo vectors.tsv ---
    vectors_file_name = 'vectors.tsv'
    vectors_file_path = os.path.join(drive_output_path, vectors_file_name)
    print(f"\nCreando archivo de vectores en Drive: {vectors_file_path}")

    try:
        # Guardar la matriz de embeddings en formato .tsv en Drive
        np.savetxt(vectors_file_path, movie_embeddings_for_projector, delimiter='\t')
        print(f"Archivo '{vectors_file_name}' guardado en Drive.")
        print(f"Tamaño del archivo de vectores: {os.path.getsize(vectors_file_path)} bytes")
    except Exception as e:
        print(f"Error al guardar el archivo de vectores en Drive: {e}")


    # --- Preparar y guardar el archivo metadata.tsv ---
    metadata_file_name = 'metadata.tsv'
    metadata_file_path = os.path.join(drive_output_path, metadata_file_name)
    print(f"\nCreando archivo de metadatos en Drive: {metadata_file_path}")

    # Seleccionar las columnas de metadatos a incluir
    metadata_columns = ['movie_title', 'genres', 'country', 'language', 'content_rating', 'imdb_score', 'title_year', 'predicted_sentiment_score']
    existing_metadata_columns = [col for col in metadata_columns if col in df_movie_metadata.columns]
    missing_metadata_columns = [col for col in metadata_columns if col not in df_movie_metadata.columns]

    if missing_metadata_columns:
        print(f"Advertencia: Las siguientes columnas de metadatos seleccionadas no se encontraron en df_movie_metadata: {missing_metadata_columns}")
        print("Solo se incluirán las columnas existentes.")
        metadata_columns_to_save = existing_metadata_columns
    else:
        metadata_columns_to_save = metadata_columns

    if not metadata_columns_to_save:
        print("Error: No hay columnas de metadatos válidas para guardar.")
    else:
        try:
            # Guardar las columnas de metadatos en formato .tsv en Drive
            df_movie_metadata[metadata_columns_to_save].to_csv(metadata_file_path, sep='\t', index=False)
            print(f"Archivo '{metadata_file_name}' guardado en Drive.")
            print(f"Tamaño del archivo de metadatos: {os.path.getsize(metadata_file_path)} bytes")
        except Exception as e:
            print(f"Error al guardar el archivo de metadatos en Drive: {e}")


    # Recomendación para TensorFlow Projector:
    # Los archivos ahora están en tu Google Drive en la ruta especificada.
    # Puedes ir a Google Drive, navegar a esa carpeta, y descargar los archivos desde allí
    # para cargarlos en https://projector.tensorflow.org/

else:
    print("\nNo se pudieron crear los archivos .tsv. Asegúrate de que las variables necesarias estén disponibles y la ruta de Drive sea válida.")

## Vista Unificada del Proceso: Datos y Relaciones

A lo largo de este ejercicio, hemos trabajado con diferentes conjuntos de datos y hemos generado nuevas representaciones y características. Podemos pensar en ellos como "tablas" conceptuales que se relacionan entre sí:

1.  **`imdb_reviews_original`**: El dataset original de reseñas de IMDb (texto y etiqueta de sentimiento).
    *   Columnas clave: `text`, `label`.
    *   Relación: Cada fila es una reseña.
2.  **`imdb_reviews_processed`**: El texto de las reseñas de IMDb después de limpieza, tokenización, eliminación de stop words y lematización.
    *   Columnas clave: `processed_text` (string de tokens lematizados).
    *   Relación: 1 a 1 con `imdb_reviews_original`.
3.  **`imdb_reviews_vectorized`**: La representación TF-IDF de las reseñas de IMDb procesadas.
    *   Representación: Matriz dispersa (ej. `tfidf_matrix_full`).
    *   Columnas conceptuales: Una columna por cada palabra/lema en el vocabulario TF-IDF.
    *   Relación: 1 a 1 con `imdb_reviews_processed`.
4.  **`sentiment_model`**: El modelo de clasificación de sentimiento entrenado (Regresión Logística).
    *   Representación: Objeto serializado (`.pkl`).
    *   Relación: Se entrena usando `imdb_reviews_vectorized` y `imdb_reviews_original['label']`. Se aplica a `df_movie_metadata_processed_keywords`.
5.  **`tfidf_vectorizer`**: El vectorizador TF-IDF entrenado.
    *   Representación: Objeto serializado (conceptual, aunque lo usamos como variable `tfidf_vectorizer_full`).
    *   Relación: Se ajusta en `imdb_reviews_processed` y se usa para transformar `imdb_reviews_processed` y `df_movie_metadata_processed_keywords`.
6.  **`df_movie_metadata_original`**: El dataset original de metadatos de Kaggle.
    *   Columnas clave: `movie_title`, `genres`, `imdb_score`, `plot_keywords`, etc.
    *   Relación: Cada fila es una película.
7.  **`df_movie_metadata_cleaned`**: El dataset de metadatos después del manejo de datos faltantes.
    *   Relación: Subconjunto de `df_movie_metadata_original` (si se eliminaron filas) o versión imputada. 1 a 1 con las filas restantes/modificadas del original. Lo representamos con la variable `df_movie_metadata`.
8.  **`df_movie_metadata_processed_keywords`**: La columna `plot_keywords` del dataset de metadatos después de limpieza y lematización.
    *   Columnas clave: `processed_plot_keywords` (string de tokens lematizados).
    *   Relación: 1 a 1 con `df_movie_metadata_cleaned`.
9.  **`df_movie_metadata_sentiment_inferred`**: El dataset de metadatos con la puntuación de sentimiento predicha añadida.
    *   Columnas clave: Todas las de `df_movie_metadata_cleaned` + `predicted_sentiment_score`.
    *   Relación: 1 a 1 con `df_movie_metadata_cleaned`. Lo representamos con la variable `df_movie_metadata` después de añadir la columna.
10. **`df_movie_metadata_vectorized_features`**: La representación vectorial combinada de las características seleccionadas del dataset de metadatos (numéricas escaladas, categóricas codificadas, sentimiento predicho).
    *   Representación: Matriz numérica (ej. `movie_features_vectorized`).
    *   Columnas conceptuales: Una columna por cada característica después del preprocesamiento (escalado, OHE).
    *   Relación: 1 a 1 con `df_movie_metadata_sentiment_inferred`.
11. **`df_movie_metadata_reduced_embeddings`**: La representación vectorial de las películas después de la reducción de dimensionalidad (t-SNE).
    *   Representación: Matriz numérica 2D o 3D (ej. `movie_embeddings_for_projector`).
    *   Columnas: Coordenadas en el espacio 2D/3D (ej. `Dim1`, `Dim2`, `Dim3`).
    *   Relación: 1 a 1 con `df_movie_metadata_vectorized_features` y `df_movie_metadata_sentiment_inferred`.

**Relación Clave para Visualización:**

Para la visualización en el TensorFlow Projector, la relación principal es entre los **embeddings reducidos (`df_movie_metadata_reduced_embeddings`)** y los **metadatos (`df_movie_metadata_sentiment_inferred`)**. Cada fila en la matriz de embeddings corresponde a la misma película en la misma fila del DataFrame de metadatos.

---

Ahora, crearemos un DataFrame de pandas que combine las columnas de metadatos clave con las coordenadas 2D/3D de t-SNE.

### Crear una Tabla Unificada (DataFrame) con Metadatos Clave y Embeddings Reducidos

In [ ]:
import pandas as pd
import numpy as np

# Asegurarnos de que df_movie_metadata y movie_embeddings_for_projector estén disponibles
try:
    if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame) and 'predicted_sentiment_score' in df_movie_metadata.columns:
         print("Utilizando el DataFrame df_movie_metadata con la columna 'predicted_sentiment_score'.")
    else:
        print("Advertencia: La variable df_movie_metadata no fue encontrada, no es un DataFrame o no tiene la columna 'predicted_sentiment_score'.")
        print("Por favor, ejecuta las celdas de carga del dataset Kaggle, manejo de faltantes (Paso 0 y Sub-etapa 1.3) y Aplicación del modelo (Sub-etapa 3.3).")
        df_movie_metadata = None # Asegurar que es None si no está disponible

    movie_embeddings_for_projector # La matriz de t-SNE 3D o 2D
    print(f"Matriz de embeddings para Projector con forma: {movie_embeddings_for_projector.shape}")

except NameError:
    print("Advertencia: La variable 'movie_embeddings_for_projector' no fue encontrada.")
    print("Por favor, ejecuta las celdas de Reducción de Dimensionalidad (Sub-etapa 5.1) primero.")
    movie_embeddings_for_projector = None


if df_movie_metadata is not None and movie_embeddings_for_projector is not None:
    # Asegurarse de que el número de filas en el DataFrame y en los embeddings coincida
    if len(df_movie_metadata) == movie_embeddings_for_projector.shape[0]:
        print("\nCombinando metadatos clave y embeddings reducidos...")

        # Seleccionar columnas de metadatos clave para la tabla unificada
        metadata_cols_for_unified_table = ['movie_title', 'genres', 'imdb_score', 'predicted_sentiment_score', 'director_name', 'actor_1_name', 'country', 'language', 'content_rating', 'title_year']
        # Asegurarse de que existan en el DataFrame
        existing_metadata_cols = [col for col in metadata_cols_for_unified_table if col in df_movie_metadata.columns]
        missing_metadata_cols = [col for col in metadata_cols_for_unified_table if col not in df_movie_metadata.columns]

        if missing_metadata_cols:
             print(f"Advertencia: Las siguientes columnas de metadatos seleccionadas para la tabla unificada no se encontraron: {missing_metadata_cols}")
             print("Solo se incluirán las columnas existentes.")


        # Crear un DataFrame con las coordenadas de los embeddings
        # Asumimos 3D para el Projector, ajustar si se usó 2D
        if movie_embeddings_for_projector.shape[1] == 3:
             df_embeddings = pd.DataFrame(movie_embeddings_for_projector, columns=['TSNE_Dim1', 'TSNE_Dim2', 'TSNE_Dim3'])
        elif movie_embeddings_for_projector.shape[1] == 2:
             df_embeddings = pd.DataFrame(movie_embeddings_for_projector, columns=['TSNE_Dim1', 'TSNE_Dim2'])
        else:
             print(f"Advertencia: Los embeddings tienen {movie_embeddings_for_projector.shape[1]} dimensiones. No se crearon columnas de TSNE con nombres estándar.")
             df_embeddings = pd.DataFrame(movie_embeddings_for_projector) # Crear DataFrame sin nombres de columna específicos

        # Combinar las columnas de metadatos seleccionadas con el DataFrame de embeddings
        # Usamos el índice para asegurar la alineación correcta
        df_unified = pd.concat([df_movie_metadata[existing_metadata_cols].reset_index(drop=True), df_embeddings.reset_index(drop=True)], axis=1)

        print("Tabla unificada (DataFrame) creada.")
        print(f"Forma de la tabla unificada: {df_unified.shape}")

        # Mostrar las primeras filas de la tabla unificada
        print("\nPrimeras filas de la tabla unificada:")
        display(df_unified.head())

        # Recomendación:
        # Este DataFrame 'df_unified' contiene la información clave de cada película
        # junto con sus coordenadas en el espacio reducido. Es útil para análisis posteriores
        # o para generar visualizaciones personalizadas fuera del TensorFlow Projector.

    else:
        print("\nError: El número de filas en df_movie_metadata y movie_embeddings_for_projector no coincide.")
        print("Asegúrate de que el preprocesamiento, vectorización y reducción de dimensionalidad se aplicaron al mismo conjunto de datos.")

else:
    print("\nNo se pudo crear la tabla unificada. Asegúrate de que df_movie_metadata y movie_embeddings_for_projector estén disponibles.")

### Exportar DataFrames a CSV en Google Drive

Guardaremos las principales tablas (DataFrames de pandas) generadas durante el proceso como archivos CSV en la carpeta especificada en Google Drive.

In [ ]:
import os
import pandas as pd # Asegurarse de que pandas esté importado

# Definir la ruta de destino en Google Drive
# Asegúrate de que Google Drive esté montado (ejecutando la celda del Paso 0)
drive_output_path = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1'

# Crear la carpeta de destino si no existe (ya deberíamos haberla creado antes, pero es seguro verificar)
if not os.path.exists(drive_output_path):
    try:
        os.makedirs(drive_output_path)
        print(f"\nCarpeta de destino en Drive creada: {drive_output_path}")
    except OSError as e:
        print(f"\nError: No se pudo crear la carpeta en Drive {drive_output_path}. Asegúrate de que Drive esté montado y la ruta sea válida.")
        drive_output_path = None # Anular la ruta si falla la creación


if drive_output_path is not None:
    print(f"\nExportando DataFrames a CSV en: {drive_output_path}")

    # --- Exportar df_movie_metadata (con score de sentimiento) ---
    try:
        if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame):
            metadata_output_filename = 'df_movie_metadata_with_sentiment.csv'
            metadata_output_path = os.path.join(drive_output_path, metadata_output_filename)
            print(f"Guardando '{metadata_output_filename}'...")
            df_movie_metadata.to_csv(metadata_output_path, index=False)
            print(f"'{metadata_output_filename}' guardado exitosamente.")
        else:
            print("Advertencia: La variable df_movie_metadata no fue encontrada o no es un DataFrame. No se exportará.")
    except Exception as e:
        print(f"Error al exportar df_movie_metadata: {e}")


    # --- Exportar la tabla unificada (df_unified) ---
    try:
        if 'df_unified' in globals() and isinstance(df_unified, pd.DataFrame):
            unified_output_filename = 'df_unified_metadata_embeddings.csv'
            unified_output_path = os.path.join(drive_output_path, unified_output_filename)
            print(f"Guardando '{unified_output_filename}'...")
            df_unified.to_csv(unified_output_path, index=False)
            print(f"'{unified_output_filename}' guardado exitosamente.")
        else:
            print("Advertencia: La variable df_unified no fue encontrada o no es un DataFrame. No se exportará.")
    except Exception as e:
        print(f"Error al exportar df_unified: {e}")

else:
    print("\nNo se pudieron exportar los DataFrames porque la ruta de Drive no es válida.")

# Recomendación:
# Ahora puedes encontrar estos archivos CSV en tu Google Drive en la ruta especificada.
# Estos archivos contienen los datos procesados y las características generadas.

### Exportar Todos los Artefactos Clave a Google Drive

Guardaremos varios resultados intermedios y finales del proceso en una subcarpeta específica en Google Drive para su posterior análisis o uso.

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle # Para guardar objetos Python como el vectorizador

# Definir la ruta base de destino en Google Drive
# Asegúrate de que Google Drive esté montado (ejecutando la celda del Paso 0)
drive_base_path = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1'
output_data_subdir = 'DataSets' # La nueva subcarpeta
drive_output_path = os.path.join(drive_base_path, output_data_subdir)

# Crear la carpeta de destino si no existe
if not os.path.exists(drive_output_path):
    try:
        os.makedirs(drive_output_path)
        print(f"\nCarpeta de destino en Drive creada: {drive_output_path}")
    except OSError as e:
        print(f"\nError: No se pudo crear la carpeta en Drive {drive_output_path}. Asegúrate de que Drive esté montado y la ruta base sea válida.")
        drive_output_path = None # Anular la ruta si falla la creación


if drive_output_path is not None:
    print(f"\nExportando artefactos de datos a: {drive_output_path}")

    # --- Artefactos relacionados con imdb_reviews ---

    # 1. Texto procesado de las reseñas (imdb_reviews_processed)
    try:
        # Necesita processed_texts_list de Sub-etapa 1.5 (celda 78c02b5d)
        if 'processed_texts_list' in globals() and isinstance(processed_texts_list, list):
            processed_reviews_filename = 'imdb_reviews_processed.csv' # Guardar como CSV
            processed_reviews_path = os.path.join(drive_output_path, processed_reviews_filename)
            print(f"Guardando '{processed_reviews_filename}'...")
            # Convertir a DataFrame para guardar fácilmente como CSV
            df_processed_reviews = pd.DataFrame({'processed_text': processed_texts_list})
            # Opcional: Añadir etiquetas si también las recolectamos en Sub-etapa 1.5 (labels_list)
            # if 'labels_list' in globals() and len(labels_list) == len(processed_texts_list):
            #     df_processed_reviews['label'] = labels_list
            df_processed_reviews.to_csv(processed_reviews_path, index=False)
            print(f"'{processed_reviews_filename}' guardado exitosamente.")
        else:
            print("Advertencia: La variable processed_texts_list no fue encontrada o no es una lista. No se exportará el texto procesado de reseñas.")
    except Exception as e:
        print(f"Error al exportar el texto procesado de reseñas: {e}")


    # 2. Matriz TF-IDF completa de las reseñas (imdb_reviews_vectorized)
    try:
        # Necesita tfidf_matrix_full de Sub-etapa 1.5 (celda 78c02b5d)
        if 'tfidf_matrix_full' in globals(): # tfidf_matrix_full es una matriz dispersa
             tfidf_matrix_filename = 'imdb_reviews_vectorized_tfidf_matrix.npz' # Formato para matrices dispersas
             tfidf_matrix_path = os.path.join(drive_output_path, tfidf_matrix_filename)
             print(f"Guardando '{tfidf_matrix_filename}'...")
             # Guardar matriz dispersa usando scipy.sparse.save_npz (necesita importar scipy)
             from scipy.sparse import save_npz
             save_npz(tfidf_matrix_path, tfidf_matrix_full)
             print(f"'{tfidf_matrix_filename}' guardado exitosamente.")
        else:
            print("Advertencia: La variable tfidf_matrix_full no fue encontrada. No se exportará la matriz TF-IDF completa.")
    except Exception as e:
        print(f"Error al exportar la matriz TF-IDF: {e}")


    # 3. Modelo de sentimiento entrenado (sentiment_model)
    # Ya lo guardamos como 'logistic_regression_sentiment_model.pkl' en la Sub-etapa 2.4
    # Podemos copiarlo o simplemente asegurarnos de que esté en la ruta correcta si lo re-guardamos.
    # Vamos a re-guardarlo en la nueva subcarpeta para consistencia.
    try:
        # Necesita model de Sub-etapa 2.2 (celda 4d393ade) o loaded_model de Sub-etapa 3.1 (celda ace7c385)
        # Usaremos la variable 'model' que es el modelo entrenado
        if 'model' in globals():
            model_filename_pkl = 'sentiment_model_logistic_regression.pkl'
            model_path_pkl = os.path.join(drive_output_path, model_filename_pkl)
            print(f"Guardando '{model_filename_pkl}'...")
            with open(model_path_pkl, 'wb') as f:
                pickle.dump(model, f)
            print(f"'{model_filename_pkl}' guardado exitosamente.")
        else:
             print("Advertencia: La variable 'model' (modelo entrenado) no fue encontrada. No se exportará el modelo.")
    except Exception as e:
         print(f"Error al exportar el modelo: {e}")


    # 4. Vectorizador TF-IDF entrenado (tfidf_vectorizer)
    try:
        # Necesita tfidf_vectorizer_full de Sub-etapa 1.5 (celda 78c02b5d)
        if 'tfidf_vectorizer_full' in globals():
            vectorizer_filename_pkl = 'tfidf_vectorizer_full.pkl'
            vectorizer_path_pkl = os.path.join(drive_output_path, vectorizer_filename_pkl)
            print(f"Guardando '{vectorizer_filename_pkl}'...")
            with open(vectorizer_path_pkl, 'wb') as f:
                pickle.dump(tfidf_vectorizer_full, f)
            print(f"'{vectorizer_filename_pkl}' guardado exitosamente.")
        else:
            print("Advertencia: La variable tfidf_vectorizer_full no fue encontrada. No se exportará el vectorizador.")
    except Exception as e:
        print(f"Error al exportar el vectorizador TF-IDF: {e}")


    # --- Artefactos relacionados con df_movie_metadata ---

    # 5. Texto procesado de las palabras clave (df_movie_metadata_processed_keywords)
    try:
        # Necesita processed_keywords_list de Sub-etapa 3.2 (celda 81b69a34)
        if 'processed_keywords_list' in globals() and isinstance(processed_keywords_list, list):
            processed_keywords_filename = 'movie_keywords_processed.csv' # Guardar como CSV
            processed_keywords_path = os.path.join(drive_output_path, processed_keywords_filename)
            print(f"Guardando '{processed_keywords_filename}'...")
            df_processed_keywords = pd.DataFrame({'processed_keywords': processed_keywords_list})
            df_processed_keywords.to_csv(processed_keywords_path, index=False)
            print(f"'{processed_keywords_filename}' guardado exitosamente.")
        else:
            print("Advertencia: La variable processed_keywords_list no fue encontrada o no es una lista. No se exportará el texto procesado de keywords.")
    except Exception as e:
        print(f"Error al exportar el texto procesado de keywords: {e}")


    # 6. DataFrame de metadatos con puntuación de sentimiento (df_movie_metadata_sentiment_inferred)
    # Ya lo guardamos como 'df_movie_metadata_with_sentiment.csv' en la celda 42af5c04
    # Vamos a re-guardarlo en la nueva subcarpeta.
    try:
        # Necesita df_movie_metadata de Sub-etapa 3.3 (celda d02357a2), que incluye la columna 'predicted_sentiment_score'
        if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame) and 'predicted_sentiment_score' in df_movie_metadata.columns:
            metadata_sentiment_filename_csv = 'df_movie_metadata_with_sentiment.csv'
            metadata_sentiment_path_csv = os.path.join(drive_output_path, metadata_sentiment_filename_csv)
            print(f"Guardando '{metadata_sentiment_filename_csv}'...")
            df_movie_metadata.to_csv(metadata_sentiment_path_csv, index=False)
            print(f"'{metadata_sentiment_filename_csv}' guardado exitosamente.")
        else:
            print("Advertencia: La variable df_movie_metadata (con score de sentimiento) no fue encontrada. No se exportará el DataFrame de metadatos con sentimiento.")
    except Exception as e:
        print(f"Error al exportar df_movie_metadata con sentimiento: {e}")


    # 7. Matriz de características vectorizadas de metadatos (df_movie_metadata_vectorized_features)
    try:
        # Necesita movie_features_vectorized de Sub-etapa 4.1 (celda 6cb10795)
        if 'movie_features_vectorized' in globals(): # movie_features_vectorized es una matriz dispersa
            metadata_features_filename = 'movie_features_vectorized.npz' # Formato para matrices dispersas
            metadata_features_path = os.path.join(drive_output_path, metadata_features_filename)
            print(f"Guardando '{metadata_features_filename}'...")
            from scipy.sparse import save_npz # Importar de nuevo por si acaso
            save_npz(metadata_features_path, movie_features_vectorized)
            print(f"'{metadata_features_filename}' guardado exitosamente.")
        else:
            print("Advertencia: La variable movie_features_vectorized no fue encontrada. No se exportará la matriz de características vectorizadas de metadatos.")
    except Exception as e:
        print(f"Error al exportar la matriz de características vectorizadas de metadatos: {e}")


    # 8. Embeddings reducidos de metadatos (df_movie_metadata_reduced_embeddings)
    # Ya lo guardamos como 'vectors.tsv' en la celda 5933356b
    # Vamos a re-guardarlo en la nueva subcarpeta.
    try:
        # Necesita movie_embeddings_for_projector de Sub-etapa 5.1 (celda e108d5db)
        if 'movie_embeddings_for_projector' in globals(): # Es un array numpy denso
             embeddings_filename_tsv = 'movie_embeddings_reduced.tsv' # Mantener formato tsv para Projector
             embeddings_path_tsv = os.path.join(drive_output_path, embeddings_filename_tsv)
             print(f"Guardando '{embeddings_filename_tsv}'...")
             np.savetxt(embeddings_path_tsv, movie_embeddings_for_projector, delimiter='\t')
             print(f"'{embeddings_filename_tsv}' guardado exitosamente.")

             # Opcional: También guardar el metadata.tsv correspondiente en la misma subcarpeta
             # Necesita df_movie_metadata de Sub-etapa 3.3 (celda d02357a2)
             if 'df_movie_metadata' in globals() and isinstance(df_movie_metadata, pd.DataFrame):
                metadata_projector_filename = 'metadata.tsv' # Mantener nombre para Projector
                metadata_projector_path = os.path.join(drive_output_path, metadata_projector_filename)
                print(f"Guardando '{metadata_projector_filename}' para Projector...")
                # Seleccionar las columnas de metadatos a incluir (mismas que en Sub-etapa 5.2)
                metadata_columns_projector = ['movie_title', 'genres', 'country', 'language', 'content_rating', 'imdb_score', 'title_year', 'predicted_sentiment_score']
                existing_metadata_columns_projector = [col for col in metadata_columns_projector if col in df_movie_metadata.columns]
                if existing_metadata_columns_projector:
                    df_movie_metadata[existing_metadata_columns_projector].to_csv(metadata_projector_path, sep='\t', index=False)
                    print(f"'{metadata_projector_filename}' guardado exitosamente.")
                else:
                    print("Advertencia: No hay columnas de metadatos válidas para guardar metadata.tsv para Projector.")
             else:
                print("Advertencia: df_movie_metadata no está disponible para guardar metadata.tsv para Projector.")

        else:
            print("Advertencia: La variable movie_embeddings_for_projector no fue encontrada. No se exportarán los embeddings reducidos.")
    except Exception as e:
        print(f"Error al exportar los embeddings reducidos o metadata.tsv: {e}")


    # 9. Tabla unificada (df_unified)
    # Ya lo guardamos como 'df_unified_metadata_embeddings.csv' en la celda 42af5c04
    # Vamos a re-guardarlo en la nueva subcarpeta.
    try:
        # Necesita df_unified de la celda 0e8ebbd5
        if 'df_unified' in globals() and isinstance(df_unified, pd.DataFrame):
            unified_filename_csv = 'df_unified_metadata_embeddings.csv'
            unified_path_csv = os.path.join(drive_output_path, unified_filename_csv)
            print(f"Guardando '{unified_filename_csv}'...")
            df_unified.to_csv(unified_path_csv, index=False)
            print(f"'{unified_filename_csv}' guardado exitosamente.")
        else:
            print("Advertencia: La variable df_unified no fue encontrada o no es un DataFrame. No se exportará la tabla unificada.")
    except Exception as e:
        print(f"Error al exportar la tabla unificada: {e}")


    print("\nProceso de exportación de artefactos de datos completado.")

else:
    print("\nNo se pudieron exportar los artefactos de datos porque la ruta de Drive no es válida.")

In [ ]:
# Nivel 1: Uso de abstracciones con conciencia de lo subyacente
from transformers import AutoModel, AutoTokenizer
import torch # PyTorch es comúnmente usado con Hugging Face Transformers
import numpy as np # Para manejar los embeddings resultantes
import pandas as pd # Para cargar el archivo CSV desde Drive
import os # Para unir rutas

# Definir la ruta del archivo CSV con el texto procesado en Google Drive
drive_data_path = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1/DataSets'
processed_reviews_filename = 'imdb_reviews_processed.csv'
processed_reviews_filepath = os.path.join(drive_data_path, processed_reviews_filename)

# --- Cargar el dataset de texto procesado desde Google Drive ---
print(f"Cargando texto procesado desde: {processed_reviews_filepath}")
try:
    # Asegurarse de que Google Drive esté montado (ejecutar la celda del Paso 0 si es necesario)
    # y que el archivo exista en la ruta especificada.
    df_processed_reviews = pd.read_csv(processed_reviews_filepath)
    print(f"Dataset '{processed_reviews_filename}' cargado exitosamente.")
    print(f"Forma del DataFrame cargado: {df_processed_reviews.shape}")

    # Extraer la columna de texto procesado
    # Asumimos que la columna se llama 'processed_text' como la guardamos
    if 'processed_text' in df_processed_reviews.columns:
        all_processed_texts = df_processed_reviews['processed_text'].tolist()
        print(f"Extraídos {len(all_processed_texts)} textos procesados.")
        # Manejar posibles valores NaN que read_csv podría haber introducido si la columna tenía vacíos
        all_processed_texts = [text if isinstance(text, str) else "" for text in all_processed_texts]

    else:
        print(f"Error: La columna 'processed_text' no fue encontrada en '{processed_reviews_filename}'.")
        all_processed_texts = [] # Usar lista vacía para evitar errores posteriores
        df_processed_reviews = None # Set df to None to indicate data issue

except FileNotFoundError:
    print(f"Error: El archivo '{processed_reviews_filepath}' no fue encontrado.")
    print("Asegúrate de que Google Drive esté montado y el archivo exista en esa ruta.")
    all_processed_texts = []
    df_processed_reviews = None
except Exception as e:
    print(f"Error al cargar o procesar el archivo CSV: {e}")
    all_processed_texts = []
    df_processed_reviews = None


if all_processed_texts:
    # --- Cargar un modelo y tokenizador pre-entrenados ---
    # Elegimos un modelo Transformer pequeño y rápido para demostración
    model_name = "distilbert-base-uncased"

    print(f"\nCargando tokenizador y modelo Transformer: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    print("Tokenizador y modelo cargados.")

    # --- Preparar una muestra de texto para obtener embeddings ---
    # Usaremos una muestra del texto cargado para evitar procesar todo el dataset grande
    sample_size_for_embeddings = 10 # Reducir el tamaño de la muestra para una ejecución más rápida
    if len(all_processed_texts) > sample_size_for_embeddings:
        sample_texts_for_embeddings = all_processed_texts[:sample_size_for_embeddings]
        print(f"\nUsando una muestra de {sample_size_for_embeddings} textos para obtener embeddings.")
    else:
        sample_texts_for_embeddings = all_processed_texts
        print(f"\nUsando todos los {len(all_processed_texts)} textos cargados para obtener embeddings.")


    print(f"Ejemplos de textos para obtener embeddings ({len(sample_texts_for_embeddings)}):")
    for i, review in enumerate(sample_texts_for_embeddings):
        print(f"Texto {i+1}: {review[:200]}...") # Mostrar solo los primeros 200 caracteres
        print("-" * 20)


    # --- Tokenizar los textos ---
    # `padding='max_length'` asegura que todas las secuencias tengan la misma longitud
    # `truncation=True` corta las secuencias si son más largas que el largo máximo del modelo
    # `return_tensors="pt"` devuelve tensores de PyTorch
    print("\nTokenizando textos...")
    # Handle potential empty strings in the sample_texts_for_embeddings list
    inputs = tokenizer(sample_texts_for_embeddings, padding=True, truncation=True, return_tensors="pt") # Use padding=True for dynamic padding


    print("Tokenización completa. Forma de los inputs (input_ids):", inputs['input_ids'].shape)


    # --- Obtener los embeddings del modelo ---
    # Mover el modelo a la GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()} # Mover inputs a la GPU

    print(f"\nObteniendo embeddings del modelo Transformer en {device}...")
    with torch.no_grad():
        outputs = model(**inputs)

    # Los embeddings de la última capa suelen estar en outputs.last_hidden_state
    # Obtener el embedding del token [CLS] para cada secuencia
    cls_embeddings = outputs.last_hidden_state[:, 0, :]

    # Convertir los embeddings de PyTorch a un array de NumPy (mover de nuevo a CPU si estaba en GPU)
    cls_embeddings_np = cls_embeddings.cpu().numpy()

    print("Obtención de embeddings completa.")
    print(f"Forma de los embeddings [CLS]: {cls_embeddings_np.shape}") # (Número de textos, Tamaño del embedding)

    print("\nPrimer embedding [CLS] (primeros 20 valores):")
    print(cls_embeddings_np[0, :20])
    print("...")


    # Recomendación para la Capa 1:
    # Ahora tienes representaciones vectoriales de alta dimensión (embeddings) para cada texto procesado de la muestra cargada.
    # Puedes usar estos embeddings como características para entrenar un clasificador (ej. Regresión Logística, SVM, etc.).
    # Para la Capa 2, podrías profundizar en:
    # - Cómo funcionan los tokenizadores (Subword tokenization).
    # - La arquitectura interna del modelo Transformer (Atención, Capas Feed-Forward).
    # - El proceso de pre-entrenamiento (Masked Language Modeling, Next Sentence Prediction).
    # - Cómo se obtienen diferentes tipos de embeddings (promedio de palabras, [CLS] token, etc.).

else:
    print("\nNo se pudo obtener embeddings porque no se cargaron textos procesados desde Google Drive.")

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd # Para manejar etiquetas

# Asegurarnos de que los embeddings del Transformer estén disponibles
try:
    # cls_embeddings_np is from the last executed cell (7a68ce72)
    cls_embeddings_np
    print(f"Embeddings del Transformer encontrados con forma: {cls_embeddings_np.shape}")
except NameError:
    print("Advertencia: La variable 'cls_embeddings_np' (embeddings del Transformer) no fue encontrada.")
    print("Por favor, ejecuta la celda anterior (la que usa Hugging Face Transformers) para generarlos.")
    cls_embeddings_np = None # Asegurar que es None si no está disponible


# Asegurarnos de tener acceso a las etiquetas para la muestra (asumiendo que el dataset original está cargado)
# NOTA: Esto asume que el orden de los embeddings en cls_embeddings_np corresponde al orden de los primeros
# 'sample_size_for_embeddings' elementos del dataset original. Esto puede no ser robusto si el preprocesamiento
# alteró el orden o si se usó una muestra diferente. Para un flujo de trabajo robusto, las etiquetas deberían
# guardarse junto con los textos procesados.
try:
    dataset # Variable del dataset original de tfds (celda 4469e4b8)
    # Tomar las etiquetas correspondientes a la muestra
    sample_size = cls_embeddings_np.shape[0] # Usar el tamaño real de la muestra de embeddings
    sample_labels = [label.numpy() for text, label in dataset.take(sample_size)]
    print(f"Etiquetas correspondientes a la muestra obtenidas ({len(sample_labels)} etiquetas).")
    sample_labels_array = np.array(sample_labels)

except NameError:
    print("Advertencia: La variable 'dataset' (dataset original de tfds) no fue encontrada.")
    print("No se pudieron obtener las etiquetas correspondientes a los embeddings de la muestra.")
    sample_labels_array = None
except Exception as e:
    print(f"Error al obtener las etiquetas de la muestra: {e}")
    sample_labels_array = None


if cls_embeddings_np is not None:
    print("\nAplicando PCA para reducción de dimensionalidad...")

    # --- Aplicar PCA a 2 dimensiones ---
    pca_2d = PCA(n_components=2, random_state=42)
    embeddings_pca_2d = pca_2d.fit_transform(cls_embeddings_np)

    print(f"PCA a 2D completo. Forma: {embeddings_pca_2d.shape}")
    # print(f"Varianza explicada por 2 componentes: {pca_2d.explained_variance_ratio_.sum():.4f}")


    # --- Aplicar PCA a 3 dimensiones ---
    pca_3d = PCA(n_components=3, random_state=42)
    embeddings_pca_3d = pca_3d.fit_transform(cls_embeddings_np)

    print(f"PCA a 3D completo. Forma: {embeddings_pca_3d.shape}")
    # print(f"Varianza explicada por 3 componentes: {pca_3d.explained_variance_ratio_.sum():.4f}")


    # --- Visualización en Colab (2D y 3D) ---
    print("\nVisualizando embeddings reducidos (muestra pequeña)...")

    # Aplicar el estilo oscuro si ya fue configurado
    plt.style.use('dark_background')

    # Visualización 2D
    plt.figure(figsize=(8, 6))
    # Si tenemos etiquetas, podemos usar colores para diferenciar clases
    if sample_labels_array is not None:
        # Cambiar colormap a uno que contraste mejor con fondo oscuro
        scatter = plt.scatter(embeddings_pca_2d[:, 0], embeddings_pca_2d[:, 1], c=sample_labels_array, cmap='viridis') # Usar 'viridis'
        plt.colorbar(scatter, label='Etiqueta (0: Negativo, 1: Positivo)')
    else:
        plt.scatter(embeddings_pca_2d[:, 0], embeddings_pca_2d[:, 1], color='skyblue') # Color por defecto si no hay etiquetas
    plt.title('PCA 2D de Embeddings del Transformer (Muestra)')
    plt.xlabel('Componente Principal 1')
    plt.ylabel('Componente Principal 2')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

    print("\nExplicación de lo que verías con un dataset más grande (Visualización 2D):")
    print("Con un conjunto de datos más grande, esperarías ver grupos (clusters) de puntos.")
    print("Los puntos que están cerca unos de otros en este espacio 2D/3D representan reseñas que el modelo Transformer considera semánticamente similares.")
    print("Si usas colores basados en la etiqueta de sentimiento (0 o 1), podrías observar si el modelo Transformer ha separado bien las reseñas positivas de las negativas en este espacio.")
    print("PCA intenta preservar la varianza global de los datos, por lo que los ejes (Componente Principal 1 y 2) representan las direcciones de mayor variabilidad en los embeddings originales.")


    # Visualización 3D (requiere instalar matplotlib con soporte 3D si no está ya)
    # from mpl_toolkits.mplot3d import Axes3D # Descomentar si es necesario

    print("\nVisualizando embeddings reducidos en 3D (muestra pequeña)...")
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    if sample_labels_array is not None:
         # Cambiar colormap a uno que contraste mejor con fondo oscuro
         scatter = ax.scatter(embeddings_pca_3d[:, 0], embeddings_pca_3d[:, 1], embeddings_pca_3d[:, 2], c=sample_labels_array, cmap='viridis') # Usar 'viridis'
         fig.colorbar(scatter, label='Etiqueta (0: Negativo, 1: Positivo)')
    else:
         ax.scatter(embeddings_pca_3d[:, 0], embeddings_pca_3d[:, 1], embeddings_pca_3d[:, 2], color='skyblue') # Color por defecto
    ax.set_title('PCA 3D de Embeddings del Transformer (Muestra)')
    ax.set_xlabel('Componente Principal 1')
    ax.set_ylabel('Componente Principal 2')
    ax.set_zlabel('Componente Principal 3')
    plt.show()

    print("\nExplicación de lo que verías con un dataset más grande (Visualización 3D):")
    print("Similar a la vista 2D, la vista 3D te permite explorar la agrupación de reseñas semánticamente similares.")
    print("Al poder rotar el gráfico, a veces se pueden identificar separaciones o estructuras que no son visibles en 2D.")
    print("Los clusters en 3D también indican grupos de reseñas con características semánticas compartidas según el modelo Transformer.")


else:
    print("\nNo se pudo realizar la reducción de dimensionalidad y visualización porque los embeddings del Transformer no están disponibles.")

In [ ]:
import os
import numpy as np
import pandas as pd

# Definir la ruta de destino en Google Drive
# Asegúrate de que Google Drive esté montado (ejecutando la celda del Paso 0)
drive_base_path = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1'
output_data_subdir = 'DataSets' # La subcarpeta donde guardaremos los archivos
drive_output_path = os.path.join(drive_base_path, output_data_subdir)

# Crear la carpeta de destino si no existe
if not os.path.exists(drive_output_path):
    try:
        os.makedirs(drive_output_path)
        print(f"\nCarpeta de destino en Drive creada: {drive_output_path}")
    except OSError as e:
        print(f"\nError: No se pudo crear la carpeta en Drive {drive_output_path}. Asegúrate de que Drive esté montado y la ruta base sea válida.")
        drive_output_path = None # Anular la ruta si falla la creación


# Asegurarnos de que los embeddings reducidos y las etiquetas estén disponibles
try:
    # embeddings_pca_3d is from the previous cell
    embeddings_pca_3d
    # sample_labels_array is from the previous cell
    sample_labels_array
    if embeddings_pca_3d is not None and sample_labels_array is not None:
        print(f"Embeddings PCA 3D encontrados con forma: {embeddings_pca_3d.shape}")
        print(f"Etiquetas de la muestra encontradas con forma: {sample_labels_array.shape}")
        # Verificar que el número de embeddings y etiquetas coincida
        if embeddings_pca_3d.shape[0] != sample_labels_array.shape[0]:
            print("Advertencia: El número de embeddings reducidos no coincide con el número de etiquetas.")
            print("No se podrán generar los archivos .tsv para el Projector con metadatos correctos.")
            embeddings_pca_3d = None # Anular si no coinciden para evitar errores
            sample_labels_array = None
    else:
         embeddings_pca_3d = None
         sample_labels_array = None


except NameError:
    print("Advertencia: Las variables 'embeddings_pca_3d' o 'sample_labels_array' no fueron encontradas.")
    print("Por favor, ejecuta la celda anterior (PCA y visualización en Colab) para generarlas.")
    embeddings_pca_3d = None
    sample_labels_array = None


if embeddings_pca_3d is not None and sample_labels_array is not None and drive_output_path is not None:

    print("\nPreparando archivos .tsv para TensorFlow Projector...")

    # --- Preparar y guardar el archivo vectors.tsv ---
    # Usaremos los embeddings 3D para el Projector
    vectors_file_name = 'transformer_pca_vectors_sample.tsv' # Nombre específico
    vectors_file_path = os.path.join(drive_output_path, vectors_file_name)
    print(f"Creando archivo de vectores en Drive: {vectors_file_path}")

    try:
        # Guardar la matriz de embeddings en formato .tsv en Drive
        np.savetxt(vectors_file_path, embeddings_pca_3d, delimiter='\t')
        print(f"Archivo '{vectors_file_name}' guardado en Drive.")
        print(f"Tamaño del archivo de vectores: {os.path.getsize(vectors_file_path)} bytes")
    except Exception as e:
        print(f"Error al guardar el archivo de vectores en Drive: {e}")


    # --- Preparar y guardar el archivo metadata.tsv ---
    metadata_file_name = 'transformer_pca_metadata_sample.tsv' # Nombre específico
    metadata_file_path = os.path.join(drive_output_path, metadata_file_name)
    print(f"\nCreando archivo de metadatos en Drive: {metadata_file_path}")

    # Para este ejemplo, los metadatos serán simplemente las etiquetas de sentimiento
    # En un caso real, podrías incluir el texto original, texto procesado, etc.
    df_metadata = pd.DataFrame({'sentiment_label': sample_labels_array})

    try:
        # Guardar el DataFrame de metadatos en formato .tsv en Drive
        # index=False para no escribir el índice de pandas como columna
        df_metadata.to_csv(metadata_file_path, sep='\t', index=False)
        print(f"Archivo '{metadata_file_name}' guardado en Drive.")
        print(f"Tamaño del archivo de metadatos: {os.path.getsize(metadata_file_path)} bytes")
    except Exception as e:
        print(f"Error al guardar el archivo de metadatos en Drive: {e}")


    print("\nArchivos .tsv para TensorFlow Projector generados y guardados.")
    print("\nRecomendación para TensorFlow Projector:")
    print(f"Los archivos '{vectors_file_name}' y '{metadata_file_name}' ahora están en tu Google Drive en la ruta: {drive_output_path}")
    print("Puedes ir a Google Drive, navegar a esa carpeta, y descargar los archivos desde allí.")
    print("Luego, ve a https://projector.tensorflow.org/ y sigue estos pasos:")
    print("1. Haz clic en el botón 'Load' en el panel izquierdo.")
    print("2. Selecciona 'Choose files'.")
    print(f"3. Carga el archivo de vectores: '{vectors_file_name}'")
    print(f"4. Carga el archivo de metadatos: '{metadata_file_name}'")
    print("\nUna vez cargados, verás los puntos en el espacio 3D (o 2D si usaste esa opción).")
    print("Puedes colorear los puntos según la columna 'sentiment_label' para ver si PCA separó las clases de sentimiento.")
    print("Con un dataset más grande, podrías explorar clusters de puntos que correspondan a temas o tipos de reseñas.")


else:
    print("\nNo se pudieron crear los archivos .tsv. Asegúrate de que los embeddings reducidos, las etiquetas y la ruta de Drive sean válidos.")

In [ ]:
# --- Imports necesarios ---
import tensorflow as tf
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np
import os
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.decomposition import PCA
import time # Importar time para posibles delays (aunque intentaremos evitarlo)


# --- Asegurar que los recursos de NLTK y stop words estén descargados ---
# Mover las descargas al inicio de la celda para mayor seguridad
print("Iniciando verificación/descarga de recursos de NLTK...")
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Descargando recurso 'punkt'...")
    nltk.download('punkt')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    print("Descargando recurso 'wordnet'...")
    nltk.download('wordnet')

try:
    nltk.data.find('corpora/omw-1.4')
except LookupError:
    print("Descargando recurso 'omw-1.4'...")
    nltk.download('omw-1.4')

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    print("Descargando recurso 'averaged_perceptron_tagger'...")
    nltk.download('averaged_perceptron_tagger')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    print("Descargando recurso 'stopwords'...")
    nltk.download('stopwords')

print("Verificación/Descarga de recursos de NLTK completada.")

# --- Forzar la carga de recursos de NLTK después de la descarga ---
# Intentar usar los recursos inmediatamente después de la descarga para forzar su carga en memoria
try:
    word_tokenize("test sentence.")
    nltk.pos_tag(["test"])
    stopwords.words('english')
    WordNetLemmatizer().lemmatize("testing")
    print("Recursos de NLTK cargados exitosamente en runtime.")
except LookupError as e:
    print(f"Error: Los recursos de NLTK no están disponibles después de la descarga. {e}")
except Exception as e:
    print(f"Error inesperado al intentar cargar recursos de NLTK: {e}")


# --- Configuración de rutas en Google Drive ---
# Asegúrate de que Google Drive esté montado (ejecutando la celda del Paso 0)
drive_base_path = '/content/drive/MyDrive/Universidad Distrital/Diplomado_AI/Semana1'
output_data_subdir = 'DataSets'
drive_output_path = os.path.join(drive_base_path, output_data_subdir)

# Crear la carpeta de destino si no existe
if not os.path.exists(drive_output_path):
    try:
        os.makedirs(drive_output_path)
        print(f"\nCarpeta de destino en Drive creada: {drive_output_path}")
    except OSError as e:
        print(f"\nError: No se pudo crear la carpeta en Drive {drive_output_path}. Asegúrate de que Drive esté montado y la ruta base sea válida.")
        drive_output_path = None # Anular la ruta si falla la creación


# --- Función de preprocesamiento de texto (limpieza, lematización, stop words) ---
# Redefinir la función para asegurar que esté disponible y use los recursos descargados
# Las variables stop_words y lemmatizer se inicializan aquí, después de la descarga/carga forzada
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNetLemmatizer, handles LookupError"""
    try:
        # Asegurar que los recursos necesarios para pos_tag estén disponibles
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN) # Default to Noun
    except LookupError:
         # print("Recursos de NLTK para POS tagging no encontrados en runtime.") # Evitar spam en output
         return wordnet.NOUN
    except IndexError:
         # print("Error de indexación en pos_tag.") # Evitar spam en output
         return wordnet.NOUN

def clean_and_lemmatize_text(text_input):
    # Convertir a string de Python si es tensor o bytes
    if isinstance(text_input, tf.Tensor):
         if tf.size(text_input) > 0 and text_input.dtype == tf.string:
             text = text_input.numpy().decode('utf-8', errors='ignore')
         else:
             return ""
    elif isinstance(text_input, bytes):
         text = text_input.decode('utf-8', errors='ignore')
     # Handle potential NaN float input if apply is used on a Series with NaNs
    elif isinstance(text_input, float) and pd.isna(text_input):
        return "" # Return empty string for NaN inputs
    else: # Asumir que ya es un string
         text = str(text_input)

    # Limpieza básica
    text = text.lower()
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenización
    try:
        tokens = word_tokenize(text)
    except LookupError:
        # print("Recurso 'punkt' de NLTK no encontrado en runtime. No se realizará tokenización.") # Evitar spam en output
        return ""

    # Eliminar Stop Words y Lematizar
    processed_tokens = []
    for word in tokens:
        # Ensure word is not empty and not a stop word
        if word and word not in stop_words:
            lemma = lemmatizer.lemmatize(word, get_wordnet_pos(word))
            processed_tokens.append(lemma)

    return " ".join(processed_tokens) # Retornar un string con los tokens unidos


# --- Cargar una MUESTRA del dataset original y procesar ---
# Usaremos una muestra para que la ejecución sea rápida
sample_size_for_processing = 1000 # Tamaño de la muestra para esta sección

print(f"\nCargando y procesando una muestra de {sample_size_for_processing} reseñas del dataset original...")

try:
    # Cargar el dataset si no está ya en memoria
    try:
        dataset # Check if dataset variable exists
        print("Utilizando el dataset 'imdb_reviews' cargado previamente.")
    except NameError:
        print("Dataset 'imdb_reviews' no encontrado. Intentando cargarlo...")
        dataset, info = tfds.load('imdb_reviews', split='train', with_info=True, as_supervised=True)
        print("Dataset 'imdb_reviews' cargado exitosamente.")

    # Procesar la muestra
    processed_texts_sample = []
    labels_sample = []

    # Iterar sobre una muestra del dataset
    for text_tensor, label_tensor in dataset.take(sample_size_for_processing):
         processed_text = clean_and_lemmatize_text(text_tensor)
         processed_texts_sample.append(processed_text)
         labels_sample.append(label_tensor.numpy())

    print(f"Procesamiento de la muestra completo. Número de reseñas procesadas: {len(processed_texts_sample)}")

except Exception as e:
    print(f"Error al cargar o procesar la muestra del dataset: {e}")
    processed_texts_sample = []
    labels_sample = []


# --- Guardar la muestra procesada con etiquetas en un CSV en Drive ---
if processed_texts_sample and drive_output_path is not None:
    sample_csv_filename = 'imdb_reviews_processed_sample_with_labels.csv'
    sample_csv_filepath = os.path.join(drive_output_path, sample_csv_filename)

    print(f"\nGuardando muestra procesada con etiquetas en Drive: {sample_csv_filepath}")

    df_processed_sample = pd.DataFrame({
        'processed_text': processed_texts_sample,
        'label': labels_sample
    })

    try:
        df_processed_sample.to_csv(sample_csv_filepath, index=False)
        print(f"Archivo '{sample_csv_filename}' guardado exitosamente en Drive.")
    except Exception as e:
        print(f"Error al guardar el archivo CSV en Drive: {e}")

else:
    print("\nNo se pudo guardar la muestra procesada en CSV. Asegúrate de que el procesamiento fue exitoso y la ruta de Drive es válida.")


# --- Cargar el CSV de muestra procesada desde Drive ---
if drive_output_path is not None:
    sample_csv_filename = 'imdb_reviews_processed_sample_with_labels.csv' # Asegurarse de usar el mismo nombre
    sample_csv_filepath = os.path.join(drive_output_path, sample_csv_filename)

    print(f"\nCargando muestra procesada con etiquetas desde Drive: {sample_csv_filepath}")
    try:
        df_loaded_sample = pd.read_csv(sample_csv_filepath)
        print(f"Dataset '{sample_csv_filename}' cargado exitosamente.")
        print(f"Forma del DataFrame cargado: {df_loaded_sample.shape}")

        # Extraer texto y etiquetas
        if 'processed_text' in df_loaded_sample.columns and 'label' in df_loaded_sample.columns:
            loaded_texts_sample = df_loaded_sample['processed_text'].tolist()
            loaded_labels_sample = df_loaded_sample['label'].tolist()
            # Manejar posibles NaN en texto si read_csv los introdujo
            loaded_texts_sample = [text if isinstance(text, str) else "" for text in loaded_texts_sample]
            print(f"Extraídos {len(loaded_texts_sample)} textos y {len(loaded_labels_sample)} etiquetas.")
        else:
             print("Error: Las columnas esperadas ('processed_text', 'label') no fueron encontradas en el CSV cargado.")
             loaded_texts_sample = []
             loaded_labels_sample = []

    except FileNotFoundError:
        print(f"Error: El archivo '{sample_csv_filepath}' no fue encontrado al intentar cargarlo.")
        loaded_texts_sample = []
        loaded_labels_sample = []
    except Exception as e:
        print(f"Error al cargar o procesar el archivo CSV desde Drive: {e}")
        loaded_texts_sample = []
        loaded_labels_sample = []

else:
    print("\nNo se pudo cargar la muestra procesada desde Drive. Ruta de Drive no válida.")
    loaded_texts_sample = []
    loaded_labels_sample = []


# --- Generar Embeddings con Transformer para la muestra cargada ---
if loaded_texts_sample:
    # Cargar un modelo y tokenizador pre-entrenados
    model_name = "distilbert-base-uncased"

    print(f"\nCargando tokenizador y modelo Transformer: {model_name}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name)
        print("Tokenizador y modelo cargados.")

        # Tokenizar los textos
        print("\nTokenizando textos para Transformer...")
        # Use padding=True for dynamic padding, truncation=True to handle long texts
        inputs = tokenizer(loaded_texts_sample, padding=True, truncation=True, return_tensors="pt")
        print("Tokenización completa. Forma de los inputs (input_ids):", inputs['input_ids'].shape)

        # Obtener los embeddings del modelo
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        print(f"\nObteniendo embeddings del modelo Transformer en {device}...")
        with torch.no_grad():
            outputs = model(**inputs)

        # Obtener el embedding del token [CLS]
        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        cls_embeddings_np = cls_embeddings.cpu().numpy()
        print("Obtención de embeddings completa.")
        print(f"Forma de los embeddings [CLS]: {cls_embeddings_np.shape}")

    except Exception as e:
        print(f"Error al cargar el modelo Transformer o generar embeddings: {e}")
        cls_embeddings_np = None
else:
    print("\nNo se pudo generar embeddings. No hay textos cargados disponibles.")
    cls_embeddings_np = None


# --- Aplicar PCA para Reducción de Dimensionalidad (a 3D para Projector) ---
if cls_embeddings_np is not None:
    print("\nAplicando PCA para reducir la dimensionalidad a 3 componentes...")
    try:
        pca_3d = PCA(n_components=3, random_state=42)
        embeddings_pca_3d = pca_3d.fit_transform(cls_embeddings_np)
        print("Reducción de dimensionalidad con PCA (3D) completa.")
        print(f"Forma de los embeddings PCA 3D: {embeddings_pca_3d.shape}")
    except Exception as e:
        print(f"Error al aplicar PCA: {e}")
        embeddings_pca_3d = None
else:
    print("\nNo se pudo aplicar PCA. Los embeddings no están disponibles.")
    embeddings_pca_3d = None


# --- Preparar y guardar archivos .tsv para TensorFlow Projector ---
if embeddings_pca_3d is not None and loaded_labels_sample and drive_output_path is not None:
    print("\nPreparando archivos .tsv para TensorFlow Projector...")

    # Asegurarse de que el número de embeddings y etiquetas coincida
    if embeddings_pca_3d.shape[0] == len(loaded_labels_sample):

        # --- Preparar y guardar el archivo vectors.tsv ---
        vectors_file_name = 'transformer_pca_vectors_sample.tsv' # Nombre específico
        vectors_file_path = os.path.join(drive_output_path, vectors_file_name)
        print(f"Creando archivo de vectores en Drive: {vectors_file_path}")
        try:
            np.savetxt(vectors_file_path, embeddings_pca_3d, delimiter='\t')
            print(f"Archivo '{vectors_file_name}' guardado exitosamente.")
        except Exception as e:
            print(f"Error al guardar el archivo de vectores en Drive: {e}")


        # --- Preparar y guardar el archivo metadata.tsv ---
        metadata_file_name = 'transformer_pca_metadata_sample.tsv' # Nombre específico
        metadata_file_path = os.path.join(drive_output_path, metadata_file_name)
        print(f"\nCreando archivo de metadatos en Drive: {metadata_file_path}")

        # Crear DataFrame de metadatos (texto original, texto procesado, etiqueta, etc.)
        # Para este ejemplo, solo incluiremos la etiqueta y el texto procesado
        df_metadata_projector = pd.DataFrame({
            'sentiment_label': loaded_labels_sample,
            'processed_text': loaded_texts_sample # Incluir texto procesado para inspección en Projector
            # Puedes añadir otras columnas si las cargas del CSV original de metadatos y las alineas
        })

        try:
            # Guardar el DataFrame de metadatos en formato .tsv en Drive
            df_metadata_projector.to_csv(metadata_file_path, sep='\t', index=False) # index=False para no escribir el índice
            print(f"Archivo '{metadata_file_name}' guardado exitosamente.")
        except Exception as e:
            print(f"Error al guardar el archivo de metadatos en Drive: {e}")

        print("\nArchivos .tsv para TensorFlow Projector generados y guardados.")
        print("\nRecomendación para TensorFlow Projector:")
        print(f"Los archivos '{vectors_file_name}' y '{metadata_file_name}' ahora están en tu Google Drive en la ruta: {drive_output_path}")
        print("Puedes ir a Google Drive, navegar a esa carpeta, y descargar los archivos desde allí.")
        print("Luego, ve a https://projector.tensorflow.org/ y sigue estos pasos:")
        print("1. Haz clic en el botón 'Load' en el panel izquierdo.")
        print("2. Selecciona 'Choose files'.")
        print(f"3. Carga el archivo de vectores: '{vectors_file_name}'")
        print(f"4. Carga el archivo de metadatos: '{metadata_file_name}'")
        print("\nUna vez cargados, verás los puntos en el espacio 3D.")
        print("Puedes colorear los puntos según la columna 'sentiment_label' para ver si PCA separó las clases de sentimiento.")
        print("Al hacer clic en un punto, podrás ver el 'processed_text' asociado en el panel lateral.")


    else:
        print("Error: El número de embeddings reducidos no coincide con el número de etiquetas cargadas.")


else:
    print("\nNo se pudieron crear los archivos .tsv. Asegúrate de que los embeddings reducidos y las etiquetas estén disponibles y la ruta de Drive sea válida.")